## set up

In [5067]:
# imports
import os
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
import json
from pydantic import BaseModel, Field
import string
import re
from typing import Optional 
import asyncio
import aiofiles
import time

In [5068]:
# Define a semaphore to limit concurrency
SEMAPHORE_LIMIT = 20
semaphore = asyncio.Semaphore(SEMAPHORE_LIMIT)

async def semaphore_wrapper(func, *args):
    """Wrap async function with semaphore to limit concurrency"""
    async with semaphore:
        return await func(*args)

In [5069]:
# api keys
try:
    with open("../secrets.json") as f:
        secrets = json.load(f)
    open_ai_key = secrets["openai"]
    anthropic_key = secrets["anthropic"]
    google_key = secrets["google"]
    os.environ["OPENAI_API_KEY"] = open_ai_key
    os.environ["ANTHROPIC_API_KEY"] = anthropic_key
    os.environ["GOOGLE_API_KEY"] = google_key
    print("API key loaded.")
except FileNotFoundError:
    print("Secrets file not found. YOU NEED THEM TO RUN THIS.")

API key loaded.


In [5070]:
## make sure all folders are created inside notebooks folder
if not os.path.exists("scripts"):
    os.makedirs("scripts")
if not os.path.exists("json"):
    os.makedirs("json")
if not os.path.exists("chapters"):
    os.makedirs("chapters")

In [5071]:
# NOT VERY RELEVANT
def format_gemini_response(res):
    output_formatted = res.content[7:-3]
    # turn into json
    output = json.loads(output_formatted)
    return output

class DotDict:
    # allow dot notation for dicts
    def __init__(self, dictionary):
        # check if dictionary is a string
        if isinstance(dictionary, str):
            # if it is a string, convert it to a dictionary
            print(dictionary)
            dictionary = json.loads(dictionary)
        for key, value in dictionary.items():
            setattr(self, key, value)

In [5072]:
model_type = "default"
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)
# llm = ChatOpenAI(model="gpt-4o", temperature=0.1)
llm2 = ChatAnthropic(model_name="claude-3-5-sonnet-20240620", temperature=0.1, max_tokens_to_sample=4096, api_key=os.getenv("ANTHROPIC_API_KEY"))

# NOT USING RIGHT NOW
# model_type = "gemini"
llm3 = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0.1)

In [5073]:
# Pydantic
class StoryInfo(BaseModel):
    """Get overall information about the story."""
    numChapters: int = Field(description="Number of chapters in the story")
    firstline: str = Field(description="First line of the story")
    lastline: Optional[str] = Field(description="Last line of the story or first volume")
    markers: list[str] = Field(description="List of markers to split the story into chapters, as formatted in the table of contents. Make sure the formatting matches the markers in the actual text (including the word 'chapter' if applicable). Keep all punctuation marks and capitalization, and use the full chapter names. If there are multiple volumes, focus on the first volume.")

def get_marker_prompt(story):
    prompt = f'''List the markers needed to split this story into chapters: {story}. Format your response like Story Info:
    class StoryInfo(BaseModel):
        """Get overall information about the story."""
        firstline: str = Field(description="First line of the story")
        lastline: Optional[str] = Field(description="Last line of the story or first volume")
        markers: list[str] = Field(description="List of markers to split the story into chapters. Keep all punctuation marks and capitalization.")
    '''
    return prompt

# structured_llm = llm2.with_structured_output(StoryInfo)
structured_llm = llm.with_structured_output(StoryInfo)

## split text into chapters

In [5074]:
# NOTE: Make sure your story file is in the scripts folder (as a txt file)
# read in txt file from scripts folder (inside notebooks folder)
og_story_name = "littlewomen" # name of the story (should match the name of the file (e.g., type "color" for color.txt)
story_name = og_story_name
analysis_type = "character" # character, theme

In [5075]:
# optional when loading a new story, but this part needs to be manually looked up
# for image, provide a url or a file path to the image
story_info = {
    "title": "The Tenant of Wildfell Hall",
    "type": "Novel",
    "author": "Anne Bronte",
    "year": "1848",
    "url": "https://www.gutenberg.org/ebooks/969",
    "image": "https://m.media-amazon.com/images/I/81kSxTutsoL._AC_UF1000,1000_QL80_.jpg"
}

In [5076]:
# if json file already exists, use story info from it instead
if os.path.exists(f"../src/data/{og_story_name}-new.json"):
    with open(f"../src/data/{og_story_name}-new.json") as f:
        data = json.load(f)
    story_info = {}
    story_info["title"] = data["title"]
    story_info["type"] = data["type"]
    if (data["type"] == "Movie"):
        story_info["director"] = data["director"]
    else:
        story_info["author"] = data["author"]
    story_info["year"] = data["year"]
    if "url" in data:
        story_info["url"] = data["url"]
    if "image" in data:
        story_info["image"] = data["image"]

    print(story_info)

{'title': 'Little Women', 'type': 'Book', 'author': 'Louisa May Alcott', 'year': '1868', 'url': 'https://www.gutenberg.org/ebooks/37106', 'image': 'https://cdn.britannica.com/04/126004-050-EC4DF54F/Dustcover-Louisa-May-Alcott-Little-Women-novel.jpg'}


In [5077]:
# read in the script
with open(f"scripts/{story_name}.txt") as f:
    story = f.read()

In [5078]:
if analysis_type == "theme":
    story_name += "_themes"
elif analysis_type == "2":
    story_name += "_2"

In [5079]:
if not os.path.exists(f"json/{story_name}"):
    os.makedirs(f"json/{story_name}")

In [5080]:
# focus on the first volume if there are multiple volumes
if "END OF VOL." in story:
    volumes = story.split("END OF VOL.")
    story = volumes[0]
elif "VOLUME II." in story:
    volumes = story.split("VOLUME II.")
    story = volumes[0]

In [5081]:
# split story into chunks
char_chunk = 120000
story_len = len(story)
print(story_len)
if story_len > char_chunk:
    # story_chunks = [story[i:i + char_chunk] for i in range(0, len(story), char_chunk)]
    # take 0:char_chunk/2 and -char_chunk/2: for the first chunk
    chunk = story[0:char_chunk//2] + story[-char_chunk//2:]
    story_chunks = [chunk]
else: # if story is less than 120k characters, use the whole story
    story_chunks = [story]
print(len(story_chunks))

1090382
1


In [5082]:
all_res = []
if not os.path.exists(f"json/{og_story_name}/summary.json"):
    # only run if summary.json does not exist (no markers yet)
    for i, chunk in enumerate(story_chunks):
        invalid_input = True
        while invalid_input: # try until valid input
            # see if exception is thrown
            try:
                # if model_type == 'gemini':
                #     prompt = get_marker_prompt(story)
                #     markers = llm3.invoke(prompt)
                #     res = format_gemini_response(markers)
                #     print(res)
                #     res = DotDict(res['Story Info'])
                # else:
                res = structured_llm.invoke(chunk)
                all_res.append(res)
                print(f"Chunk {i}")
                invalid_input = False
            except Exception as e:
                print("Exception thrown. Trying again.")
                print("Error:", e)
                invalid_input = True
        # if i < len(story_chunks) - 1:
        #     # Pause for a specified time (e.g., 60 seconds) to avoid rate limits
        #     time.sleep(60)  # Adjust the sleep duration as per your rate limit requirements

In [5083]:
if not os.path.exists(f"json/{og_story_name}/summary.json"):
    # save first line from first chunk, last line from last chunk, and combine markers
    first_line = all_res[0].firstline
    last_line = all_res[-1].lastline
    num_chapters = sum([res.numChapters for res in all_res])
    markers = []
    for res in all_res:
        markers.extend(res.markers)

    num_chapters, markers

In [5084]:
if not os.path.exists(f"json/{og_story_name}/summary.json"):
    # checking output
    first_line, last_line

In [5085]:
# Format the story
story = re.sub(r' {2,}', ' ', story)  # Replace consecutive spaces with one space
story = re.sub(r'\n{2,}', '\n', story)  # Replace consecutive newlines with one newline
story = re.sub(r' ?\n ?', '\n', story)  # Replace space + newline or newline + space with one newline
story = story.replace("“", "\"").replace("”", "\"").replace("‘", "'").replace("’", "'")

In [5086]:
story_formatted = story.replace("\n", " ")
# replace weird quotes too
story_formatted = story_formatted.replace("\"", " ").replace("'", " ")

In [5087]:
if not os.path.exists(f"json/{og_story_name}/summary.json"):
    # remove any marker that doesn't include one of these words (may or may not be necessary)
    exclude_words = ["part", "preface", "contents", "introduction", "end", "epilogue", "letter", "volume"]
    markers = [m for m in markers if not any(word in m.lower() for word in exclude_words)]

In [5088]:
if not os.path.exists(f"json/{og_story_name}/summary.json"):
    # save as json if it doesn't exist yet
    story_json = {
        "first_line": first_line,
        "last_line": last_line,
        "markers": markers
    }

    if not os.path.exists(f"json/{og_story_name}"):
        os.makedirs(f"json/{og_story_name}")
    with open(f"json/{og_story_name}/summary.json", "w") as f:
        json.dump(story_json, f, indent=4)

In [5089]:
# read in markers from summary.json
with open(f"json/{og_story_name}/summary.json") as f:
    summary = json.load(f)
    markers = summary["markers"]
    first_line = summary["first_line"]
    last_line = summary["last_line"]

In [5090]:
story_formatted[-100:] 

'help produce our new eBooks, and how to subscribe to our email newsletter to hear about new eBooks. '

In [5091]:
first_ind = story_formatted.find(first_line.replace("\"", " ").replace("'", " "))
first_ind, first_line.replace("\"", " ").replace("'", " ")

(9648,
 ' Christmas won t be Christmas without any presents,  grumbled Jo, lying on the rug.')

In [5092]:
second_marker = markers[1] + " "
last_line_index = story_formatted.find(second_marker, first_ind)
last_line_index, second_marker, story_formatted[last_line_index-50:last_line_index+50]

(32181,
 'II. A MERRY CHRISTMAS. ',
 ' sung as usual] [Illustration: A Merry Christmas] II. A MERRY CHRISTMAS. Jo was the first to wake in')

In [5093]:
last_ind = story_formatted.find(last_line.replace("\"", " ").replace("'", " "))
last_ind, last_line.replace("\"", " ").replace("'", " ")

(1027174,
 ' O, my girls, however long you may live, I never can wish you a greater happiness than this! ')

In [5094]:
# crop story to first_ind and last_ind
story = story[first_ind:last_ind + len(last_line)]

In [5095]:
# crop story_formatted too
story_formatted = story_formatted[first_ind:last_ind + len(last_line)]

In [5096]:
story[-100:], story_formatted[-100:]

('ity,--\n"O, my girls, however long you may live, I never can wish you a greater\nhappiness than this!"',
 'ity,--  O, my girls, however long you may live, I never can wish you a greater happiness than this! ')

In [5097]:
# Only run if chapters folder does not exist yet
if not os.path.exists(f"chapters/{og_story_name}"):
    # split story into chapters based on markers
    chapters = []
    cur_first_line = first_line.replace("\"", " ").replace("'", " ")
    cur_min_index = 0
    for i, marker in enumerate(markers):
        # print("marker", marker)
        exists_next = i+1 < len(markers)
        next_marker = markers[i+1] if exists_next else ""
        # print("next_marker", next_marker)
        
        if exists_next and " ACT " in next_marker.upper():
            next_marker = next_marker.upper()
        formatted_last_line = next_marker if exists_next else last_line
        # print("formatted_last_line", formatted_last_line)

        if " ACT " not in next_marker.upper():
            formatted_last_line += " "
        cur_last_line = formatted_last_line
        cur_last_line = cur_last_line.replace("\"", " ").replace("'", " ")
        first_line_index = story_formatted.find(cur_first_line, cur_min_index)
        print("cur_first_line", repr(cur_first_line))
        print("first_line_index", first_line_index)
        print("Snippet around first_line_index:", repr(story_formatted[first_line_index:first_line_index + 200]))

        if i != 0 and " ACT " not in marker.upper():
            first_line_index += len(cur_first_line)
        last_line_index = story_formatted.find(cur_last_line, first_line_index)
        print("cur_last_line", repr(cur_last_line))
        print("last_line_index", last_line_index)
        print("Snippet around last_line_index:", repr(story_formatted[last_line_index-200:last_line_index]))
        
        if i+1 == len(markers):
            if last_line_index == -1:
                last_line_index = len(story_formatted)
            else:
                last_line_index += len(cur_last_line) 
        chapter = story[first_line_index:last_line_index].strip()

        # add line number in front of each line
        chapter = chapter.split("\n")
        for j, line in enumerate(chapter):
            chapter[j] = f"LINE {j+1}: {line}"
        chapter = "\n".join(chapter)
        chapters.append(chapter)
        cur_first_line = cur_last_line
        cur_min_index = last_line_index

In [5098]:
if not os.path.exists(f"chapters/{og_story_name}"):
    # check results
    for i, marker in enumerate(markers):
        print(f"Chapter {marker}")
        chapter_first_line = chapters[i][:100]
        chapter_last_line = chapters[i][-100:]
        print("first_line:", chapter_first_line)
        print("last_line:", chapter_last_line)
        print("\n")

In [5099]:
if not os.path.exists(f"chapters/{og_story_name}"):
    # create folder for story if it doesn't exist
    os.makedirs(f"chapters/{og_story_name}", exist_ok=True)

    # also make chapter folder in public directory
    os.makedirs(f"../public/chapters/{og_story_name}", exist_ok=True)

    # save chapters to new txt files
    for i, marker in enumerate(markers):
        with open(f"chapters/{og_story_name}/{marker}.txt", "w") as f:
            f.write(chapters[i])
            print(f"Chapter {marker} saved.")
        # save to public folder too
        with open(f"../public/chapters/{og_story_name}/{marker}.txt", "w") as f:
            f.write(chapters[i])
            print(f"Chapter {marker} saved to public folder.")

In [5100]:
# read in chapters from txt files
chapters = []
for i, marker in enumerate(markers):
    with open(f"chapters/{og_story_name}/{marker}.txt") as f:
        chapter = f.read()
        chapters.append(chapter)

In [5101]:
chapters[0]

'LINE 1: "Christmas won\'t be Christmas without any presents," grumbled Jo, lying\nLINE 2: on the rug.\nLINE 3: "It\'s so dreadful to be poor!" sighed Meg, looking down at her old\nLINE 4: dress.\nLINE 5: "I don\'t think it\'s fair for some girls to have plenty of pretty things,\nLINE 6: and other girls nothing at all," added little Amy, with an injured\nLINE 7: sniff.\nLINE 8: "We\'ve got father and mother and each other," said Beth contentedly,\nLINE 9: from her corner.\nLINE 10: The four young faces on which the firelight shone brightened at the\nLINE 11: cheerful words, but darkened again as Jo said sadly,--\nLINE 12: "We haven\'t got father, and shall not have him for a long time." She\nLINE 13: didn\'t say "perhaps never," but each silently added it, thinking of\nLINE 14: father far away, where the fighting was.\nLINE 15: Nobody spoke for a minute; then Meg said in an altered tone,--\nLINE 16: "You know the reason mother proposed not having any presents this\nLINE 17: Christmas w

## analyze scene

In [5102]:
# check if all.json exists, this will determine if we need to generate all scene details or just character details
scene_details_exist = os.path.exists(f"json/{og_story_name}/all.json")
scene_details_exist

False

In [5103]:
scene_type = "location" # location, character, or n/a

In [5104]:
extra_scene_instruction = "Break this chapter into scenes, starting at line 1."
if scene_type == "location":
    extra_scene_instruction += " Do not start a new scene until the location changes. Record the line number when this happens."
elif scene_type == "character":
    extra_scene_instruction += " Do not start a new scene until a new character enters or an old character leaves. Record the line number when this happenes."

extra_scene_instruction += "\nYour goal is to have as few scenes as possible.\nChapter text:"

extra_scene_instruction

'Break this chapter into scenes, starting at line 1. Do not start a new scene until the location changes. Record the line number when this happens.\nYour goal is to have as few scenes as possible.\nChapter text:'

In [5105]:
# Pydantic
class SceneInfo(BaseModel):
    """Get information about a scene in the story."""
    scene: str = Field(description="Short title of scene (don't include chapter name or number). Each scene should have a unique title, and each title should be no longer than 6 words.")
    summary: str = Field(description="1 line summary of the scene")
    firstline: int = Field(description="First line number of the current scene in this chapter (should be after last line number of previous scene)")
    lastline: int = Field(description="Last line number of the current scene in this chapter (should be after first line number of current scene)")
    explanation: str = Field(description="Explanation of why a new scene starts here. For the first scene in the chapter, just write 'Start of chapter'.")

class SceneListInfo(BaseModel):
    """Get overall information about the story."""
    scenelist: list[SceneInfo] = Field(description="List of key scenes in this chapter")

def get_scenelist_prompt(chapter):
    scene_prompt = f'''List the key scenes in this chapter: {chapter}. Format your response like SceneListInfo:
                    class SceneInfo(BaseModel):
                        """Get information about a scene in the story."""
                        scene: str = Field(description="Title of scene (don't include chapter name or number)")
                        summary: str = Field(description="1 line summary of the scene")
                        firstline: int = Field(description="First line number of the current scene in this chapter (should be after last line number of previous scene)")
                        lastline: int = Field(description="Last line number of the current scene in this chapter (should be after first line number of current scene)")

                    class SceneListInfo(BaseModel):
                        """Get overall information about the story."""
                        scenelist: list[SceneInfo] = Field(description="List of key scenes in this chapter")'''
    return scene_prompt

scene_llm = llm.with_structured_output(SceneListInfo)
# scene_llm = llm2.with_structured_output(SceneListInfo)

In [5106]:
# Pydantic
class CharacterInfo(BaseModel):
    """Get information about a character in this scene."""
    name: str = Field(description="Full name of the character. Character name should be 5 words or less.")
    # role: str = Field(description="Main actions and/or motivations of the character in this scene")
    importance: float = Field(description="Importance of the character in this scene from from 0: not important at all to 1: very important character")
    importance_exp: str = Field(description="Brief, 1-phrase explanation your importance rating for this character")
    emotion: str = Field(description="Emotion of the character in this scene, described in a few words or a short phrase")
    sentiment: float = Field(description="Sentiment of the character in this scene from -1 (very negative) to 1 (very positive)")
    quote: str = Field(description="A direct quote from the character in this scene to illustrate their emotions and/or actions")
    fake_quote: str = Field(description="A brief explanation of the character's emotions and/or actions in this scene if you cannot find a direct quote in the text. If you can find a direct quote, leave this field empty.")

# Pydantic
class ThemeInfo(BaseModel):
    """Get information about a theme in this scene."""
    name: str = Field(description="Short general phrase to describe the theme. Theme name should be 3 words or less.")
    # role: str = Field(description="Short description of the role of the theme in this scene")
    importance: float = Field(description="Importance of the theme in this scene from from 0: not important at all to 1: very important theme")
    importance_exp: str = Field(description="Brief, 1-phrase explanation of your importance rating for this theme")
    emotion: str = Field(description="Emotion associated with the theme in this scene, described in a few words or a short phrase")
    sentiment: float = Field(description="Sentiment of the theme in this scene from -1 (very negative) to 1 (very positive)")
    quote: str = Field(description="A direct quote in this scene to illustrate the theme")
    fake_quote: str = Field(description="A brief explanation of the theme in this scene if you cannot find a direct quote in the text. If you can find a direct quote, leave this field empty.")

class SceneDetails(BaseModel):
    """Get overall information about the scene."""
    location: str = Field(description="Physical location of the scene. Location name should be 5 words or less.")
    importance: float = Field(description="Importance of the scene from 0: not important at all to 1: very important and pivotal scene")
    conflict: float = Field(description="Conflict in the scene from 0: no conflict to 1: high conflict")
    characters: list[CharacterInfo] = Field(description="List of key characters in the scene. Make sure there are no duplicates, and focus on speaking characters. Don't include any unnamed characters.")

class SceneDetailsBare(BaseModel):
    """Get overall information about the scene."""
    characters: list[CharacterInfo] = Field(description="List of key characters in the scene. Make sure there are no duplicates, and focus on speaking characters. Don't include any unnamed characters.")

class SceneDetailsTheme(BaseModel):
    """Get overall information about the scene."""
    location: str = Field(description="Physical location of the scene. Location name should be 5 words or less.")
    importance: float = Field(description="Importance of the scene from 0: not important at all to 1: very important and pivotal scene")
    conflict: float = Field(description="Conflict in the scene from 0: no conflict to 1: high conflict")
    themes: list[ThemeInfo] = Field(description="List of key themes in the scene. Make sure there are no duplicates but try to make the themes as general as possible.")

class SceneDetailsBareTheme(BaseModel):
    """Get overall information about the scene."""
    themes: list[ThemeInfo] = Field(description="List of key themes in the scene. Make sure there are no duplicates but try to make the themes as general as possible.")

def get_scenedetails_prompt(scene):
    scene_prompt = f'''List the key details in this scene: {scene}. Format your response like SceneDetails:
                    class CharacterInfo(BaseModel):
                        """Get information about a character in this scene."""
                        name: str = Field(description="Full name of the character. Character name should be 5 words or less.")
                        role: str = Field(description="Main actions and/or motivations of the character in this scene")
                        importance: float = Field(description="Importance of the character in this scene from from 0: not important at all to 1: very important character")
                        emotion: str = Field(description="Emotion of the character in this scene, described in a few words or a short phrase")
                        sentiment: float = Field(description="Sentiment of the character in this scene from -1 (very negative) to 1 (very positive)")
                        quote: str = Field(description="A direct quote from the character in this scene to illustrate their emotions and/or actions")

                    class SceneDetails(BaseModel):
                        """Get overall information about the scene."""
                        location: str = Field(description="Physical location of the scene. Location name should be 5 words or less.")
                        importance: float = Field(description="Importance of the scene from 0: not important at all to 1: very important and pivotal scene")
                        conflict: float = Field(description="Conflict in the scene from 0: no conflict to 1: high conflict")
                        characters: list[CharacterInfo] = Field(description="List of key characters in the scene. Make sure there are no duplicates, and focus on speaking characters.  Don't include any unnamed characters.")'''
    return scene_prompt

scene_details_llm = llm.with_structured_output(SceneDetails)
scene_details_llm_bare = llm.with_structured_output(SceneDetailsBare)
scene_details_llm_theme = llm.with_structured_output(SceneDetailsTheme)
scene_details_llm_bare_theme = llm.with_structured_output(SceneDetailsBareTheme)
# scene_details_llm = llm2.with_structured_output(SceneDetails)

In [5107]:
async def get_scene_info_async(chapters, chapter_num):
    print(f"Splitting Chapter {markers[chapter_num]} into scenes...")

    cur_chapter = chapters[chapter_num]

    invalid_input = True
    while invalid_input: # try until valid input
        # see if exception is thrown
        try:
            if model_type == "gemini":
                prompt = get_scenelist_prompt(cur_chapter)
                chapter_scenes = await llm3.ainvoke(prompt)
                chapter_scenes = format_gemini_response(chapter_scenes)
            else:
                chapter_scenes = await scene_llm.ainvoke(extra_scene_instruction + "\n" + cur_chapter)
            invalid_input = False
        except Exception as e:
            print("Exception thrown. Trying again.")
            print("Error:", e)
            invalid_input = True
            
    if model_type == "gemini":
        chapter_scenes_list = [DotDict(scene) for scene in chapter_scenes['scenelist']]
    else: 
        chapter_scenes_list = chapter_scenes.scenelist
    formatted_chapter_scenes = []

    cur_last_line = 0

    for scene in chapter_scenes_list:
        scene_title = scene.scene
        scene_summary = scene.summary
        scene_first_line = scene.firstline
        scene_last_line = scene.lastline
        scene_exp = scene.explanation

        print(f"FIRST LINE: {scene_first_line}")
        print(f"LAST LINE: {scene_last_line}")
        print()

        formatted_scene = {
            "title": scene_title,
            "summary": scene_summary,
            "chapter": markers[chapter_num],
            "first_line": scene_first_line,
            "last_line": scene_last_line,
            "explanation": scene_exp
        }

        if scene_last_line > scene_first_line and scene_first_line > cur_last_line:
            # only add scene if last_line > first_line and first_line > cur_last_line
            formatted_chapter_scenes.append(formatted_scene)
            formatted_chapter = cur_chapter.replace("\n", " ")
            # replace weird quotes too
            formatted_chapter = formatted_chapter.replace("\"", " ").replace("'", " ")
            cur_last_line = scene_last_line

    # sort scenes by first_line
    formatted_chapter_scenes = sorted(formatted_chapter_scenes, key=lambda x: x["first_line"])

    # split chapter into scenes based on formatted_chapter_scenes
    scenes = []
    cur_min_index = 0
    for i, scene in enumerate(formatted_chapter_scenes):
        cur_first_line = f"LINE {scene['first_line']}:"
        # cur_first_line = formatted_chapter_scenes[i]["first_line"].strip("...").replace("\"", " ").replace("'", " ")
        first_line_index = formatted_chapter.find(cur_first_line, cur_min_index)
        if first_line_index == -1:
            # see if first_line_index overlaps with last_line of previous scene
            other_possible_first_index = formatted_chapter.find(cur_first_line)
            if other_possible_first_index != -1:
                overlap = cur_chapter[other_possible_first_index:cur_min_index]
                print(f"SCENE {i}")
                print("\nPREVIOUS SCENE")
                print(scenes[i-1])
                print("\nOVERLAP")
                print(overlap)
                print()
            first_line_index = cur_min_index
        cur_last_line = f"LINE {scene['last_line'] + 1}:"
        # cur_last_line = formatted_chapter_scenes[i]["last_line"].strip("...").replace("\"", " ").replace("'", " ")
        last_line_index = formatted_chapter.find(cur_last_line, first_line_index)
        # last_line_index += len(cur_last_line)
        if last_line_index == -1:
            last_line_index = len(cur_chapter)
        scene_text = cur_chapter[first_line_index:last_line_index].strip()
        # remove line numbers
        scene_text = re.sub(r'LINE \d+: ', '', scene_text)
        scenes.append(scene_text)
        cur_min_index = last_line_index

    # add scene text to formatted_chapter_scenes
    for i, scene in enumerate(formatted_chapter_scenes):
        formatted_chapter_scenes[i]["text"] = scenes[i]

    return scenes, formatted_chapter_scenes

In [5108]:
async def get_scene_details_async(scenes, formatted_chapter_scenes, chapter_num):
    print(f"Generating scene details for Chapter {markers[chapter_num]}...")
    all_scene_details = []

    async def process_scene(scene_text, cur_scene):
        """Async function to analyze a single scene"""
        if len(scene_text) == 0:
            return
        
        start_prompt = """
                          When analyzing this scene, focus on characters with a significant role in the scene.
                          If you can't determine the character's role or emotion, don't include them in the analysis.

                          Make sure to only use direct, continuous quotes from the text.
                          Don't paraphrase or change the text in any way (e.g., don't join multiple quotes together).
                          If the character speaks, the quote should be something they say.
                          If the character is only described, the quote should be a description of the character's actions or emotions
                          (or any quote that mentions them, even if it is said by another character).
                          Try your best to find a continous quote for each character, but if you can't find a quote, 
                          (1) set the "quote" field to "No quote available" and (2) in the "fake_quote" field,
                          write a brief explanation of the character's emotions and actions in the scene.
                          Leave the "fake_quote" field empty if you can find a continous quote.
                        
                          Scene text:

                          """
        if analysis_type == "theme":
            start_prompt = """
                            When analyzing this scene, focus on themes with a significant role in the scene.
                            If you can't determine the theme's role or emotion, don't include it in the analysis.

                            Make sure to only use direct, continuous quotes from the text that describe the theme or relate to it.
                            Don't paraphrase or change the text in any way (e.g., don't join multiple quotes together).
                            Try your best to find a continuous quote for each theme, but if you can't find a quote,
                            (1) set the "quote" field to "No quote available" and (2) in the "fake_quote" field,
                            write a brief explanation of the theme in this scene.
                            Leave the "fake_quote" field empty if you can find a continuous quote.
                        
                            Scene text:

                            """
            
        invalid_input = True
        while invalid_input:
            try:
                if analysis_type == "theme":
                    scene_details = await scene_details_llm_theme.ainvoke(start_prompt + cur_scene)
                else:
                    scene_details = await scene_details_llm.ainvoke(start_prompt + cur_scene)
                invalid_input = False
                # print(scene_details)
            except Exception as e:
                print("Exception thrown. Trying again.")
                print("Error:", e)
                invalid_input = True
        
        return scene_details

    # Run all scene details in parallel
    tasks = [process_scene(formatted_chapter_scenes[i]["text"], scenes[i]) for i in range(len(scenes))]
    all_scene_details = await asyncio.gather(*tasks)

    # add details to formatted_chapter_scenes
    for i, scene in enumerate(formatted_chapter_scenes):
        scene_text = formatted_chapter_scenes[i]["text"]
        text_length = len(scene_text)
        num_lines = formatted_chapter_scenes[i]["last_line"] - formatted_chapter_scenes[i]["first_line"] + 1
        # first_line = formatted_chapter_scenes[i]["first_line"]
        # first_line_length = len(first_line)
        # last_line = formatted_chapter_scenes[i]["last_line"]
        # last_line_length = len(last_line)

        # if text_length == 0 or text_length < first_line_length + last_line_length:
        if text_length == 0:
            # no scene text or scene text is too short
            continue
        
        cur_scene_details = all_scene_details[i]
        formatted_chapter_scenes[i]["length"] = text_length
        formatted_chapter_scenes[i]["num_lines"] = num_lines
        formatted_chapter_scenes[i]["location"] = cur_scene_details.location
        formatted_chapter_scenes[i]["importance"] = cur_scene_details.importance
        formatted_chapter_scenes[i]["conflict"] = cur_scene_details.conflict
        formatted_character_info = []

        characters = cur_scene_details.themes if analysis_type == "theme" else cur_scene_details.characters
        # if model_type == "gemini":
        #     characters = [DotDict(character) for character in characters]
        for character in characters:
            character_info = {
                "name": character.name,
                "importance": character.importance,
                "importance_exp": character.importance_exp,
                "emotion": character.emotion,
                "sentiment": character.sentiment,
                "quote": character.quote,
                "fake_quote": character.fake_quote
            }
            # add character only if emotion is not N/A and quote is not "No quote available"
            if not (character.emotion == "N/A" and "No quote available" in character.quote):
                formatted_character_info.append(character_info)
        formatted_chapter_scenes[i]["characters"] = formatted_character_info

    return formatted_chapter_scenes

In [5109]:
async def replace_characters_only_async():
    # read in data from all.json
    with open(f"json/{og_story_name}/all.json") as f:
        scenes = json.load(f)
    
    async def process_scene(scene_info, scene_text):
        """Async function to replace scene details with characters only"""

        if len(scene_text) == 0:
            return
        
        start_prompt = """
                          When analyzing this scene, focus on characters with a significant role in the scene.
                          If you can't determine the character's role or emotion, don't include them in the analysis.

                          Make sure to only use direct, continuous quotes from the text.
                          Don't paraphrase or change the text in any way (e.g., don't join multiple quotes together).
                          If the character speaks, the quote should be something they say.
                          If the character is only described, the quote should be a description of the character's actions or emotions
                          (or any quote that mentions them, even if it is said by another character).
                          Try your best to find a continous quote for each character, but if you can't find a quote, 
                          (1) set the "quote" field to "No quote available" and (2) in the "fake_quote" field,
                          write a brief explanation of the character's emotions and actions in the scene.
                          Leave the "fake_quote" field empty if you can find a continous quote.
                        
                          Scene text:

                          """
        if analysis_type == "theme":
            start_prompt = """
                            When analyzing this scene, focus on themes with a significant role in the scene.
                            If you can't determine the theme's role or emotion, don't include it in the analysis.

                            Make sure to only use direct, continuous quotes from the text that describe the theme or relate to it.
                            Don't paraphrase or change the text in any way (e.g., don't join multiple quotes together).
                            Try your best to find a continuous quote for each theme, but if you can't find a quote,
                            (1) set the "quote" field to "No quote available" and (2) in the "fake_quote" field,
                            write a brief explanation of the theme in this scene.
                            Leave the "fake_quote" field empty if you can find a continuous quote.
                        
                            Scene text:

                            """
            
        invalid_input = True
        while invalid_input:
            try:
                if analysis_type == "theme":
                    scene_details = await scene_details_llm_bare_theme.ainvoke(start_prompt + scene_text)
                else:
                    scene_details = await scene_details_llm_bare.ainvoke(start_prompt + scene_text)
                invalid_input = False
                # print(scene_details)
            except Exception as e:
                print("Exception thrown. Trying again.")
                print("Error:", e)
                invalid_input = True
            
        characters = scene_details.themes if analysis_type == "theme" else scene_details.characters
        # if model_type == "gemini":
        #     characters = [DotDict(character) for character in characters]
        new_characters = []
        for character in characters:
            character_info = {
                "name": character.name,
                "importance": character.importance,
                "importance_exp": character.importance_exp,
                "emotion": character.emotion,
                "sentiment": character.sentiment,
                "quote": character.quote,
                "fake_quote": character.fake_quote
            }
            # add character only if emotion is not N/A and quote is not "No quote available"
            if not (character.emotion == "N/A" and "No quote available" in character.quote):
                new_characters.append(character_info)
        scene_info["characters"] = new_characters

        print(f"Processed Chapter {scene_info['chapter']}, Scene {scene_info['title']}.")
        
        return scene_info

    # Run all scene details in parallel
    tasks = [semaphore_wrapper(process_scene, scenes[i], scenes[i]["text"]) for i in range(len(scenes))]
    all_scene_details = await asyncio.gather(*tasks)

    # save to all.json
    with open(f"json/{story_name}/all.json", "w") as f:
        json.dump(all_scene_details, f, indent=4)
    print(f"Scene data updated with new {analysis_type} data. Saved to all.json.")

In [5110]:
async def process_all_chapters_async():
    if scene_details_exist:
        print(f"Scene details already exist. Only replacing {analysis_type} info.")
        await replace_characters_only_async()
    else:
        # Step 1: Split each chapter into scenes in parallel
        chapter_tasks = [get_scene_info_async(chapters, i) for i in range(len(chapters))]
        chapter_results = await asyncio.gather(*chapter_tasks)

        # extract scenes and formatted_chapter_scenes
        scenes = [result[0] for result in chapter_results]
        formatted_chapter_scenes = [result[1] for result in chapter_results]

        # Step 2: Get scene details for each scene in parallel
        detail_tasks = [semaphore_wrapper(get_scene_details_async, scenes[i], formatted_chapter_scenes[i], i) for i in range(len(chapters))]
        final_formatted_scenes = await asyncio.gather(*detail_tasks)

        # Step 3: Filter and save results
        for i, marker in enumerate(markers):
            formatted_chapter_scenes = final_formatted_scenes[i]

            # remove scenes with no text
            formatted_chapter_scenes = [scene for scene in formatted_chapter_scenes if scene["text"] != "" and "characters" in scene]
            # remove characters with no quote
            for scene in formatted_chapter_scenes:
                scene["characters"] = [character for character in scene["characters"] if "(" not in character["quote"] != "" and "<" not in character["name"] and "unnamed" not in character["name"].lower()]
                # print(scene)
            
            # remove scenes with no characters
            formatted_chapter_scenes = [scene for scene in formatted_chapter_scenes if len(scene["characters"]) > 0]

            # save as json
            with open(f"json/{story_name}/{marker}.json", "w") as f:
                json.dump(formatted_chapter_scenes, f, indent=4)

            print(f"All details for Chapter {marker} saved.\n")

In [5111]:
start_time = time.time()

In [5112]:
# RUN MAIN SCENE LOOP ASYNC that will get all the scene/character/etc. info for each chapter
await process_all_chapters_async()

Splitting Chapter I. PLAYING PILGRIMS. into scenes...
Splitting Chapter II. A MERRY CHRISTMAS. into scenes...
Splitting Chapter III. THE LAURENCE BOY. into scenes...
Splitting Chapter IV. BURDENS. into scenes...
Splitting Chapter V. BEING NEIGHBORLY. into scenes...
Splitting Chapter VI. BETH FINDS THE PALACE BEAUTIFUL. into scenes...
Splitting Chapter VII. AMY'S VALLEY OF HUMILIATION. into scenes...
Splitting Chapter VIII. JO MEETS APOLLYON. into scenes...
Splitting Chapter IX. MEG GOES TO VANITY FAIR. into scenes...
Splitting Chapter X. THE P. C. AND P. O. into scenes...
Splitting Chapter XI. EXPERIMENTS. into scenes...
Splitting Chapter XII. CAMP LAURENCE. into scenes...
Splitting Chapter XIII. CASTLES IN THE AIR. into scenes...
Splitting Chapter XIV. SECRETS. into scenes...
Splitting Chapter XV. A TELEGRAM. into scenes...
Splitting Chapter XVI. LETTERS. into scenes...
Splitting Chapter XVII. LITTLE FAITHFUL. into scenes...
Splitting Chapter XVIII. DARK DAYS. into scenes...
Splitting

In [5113]:
# rank each scene by importance
# and within each scene, the characters
if not scene_details_exist:
    for i, marker in enumerate(markers):
        print(marker)
        with open(f"json/{story_name}/{marker}.json") as f:
            scenes = json.load(f)
        importances = []
        conflicts = []
        
        # extract importance from each scene
        for j, scene in enumerate(scenes):
            importances.append((j, scene["importance"]))
            conflicts.append((j, scene["conflict"]))
            # add number of each scene
            scenes[j]["number"] = j+1
        # sort importances
        sorted_importances = sorted(importances, key=lambda x: x[1], reverse=True)
        # add importance_rank to each scene
        for k, (j, _) in enumerate(sorted_importances):
            scenes[j]["importance_rank"] = k+1
        # sort conflicts
        sorted_conflicts = sorted(conflicts, key=lambda x: x[1], reverse=True)
        # add conflict_rank to each scene
        for k, (j, _) in enumerate(sorted_conflicts):
            scenes[j]["conflict_rank"] = k+1
        
        # print results
        # for j, scene in enumerate(scenes):
        #     print(scenes[j]["title"], scenes[j]["importance_rank"])
        #     print("--------------------------------")
        #     for k, character in enumerate(scene["characters"]):
        #         print(character["name"], character["importance_rank"])
        #     print()

        # save as json
        with open(f"json/{story_name}/{marker}.json", "w") as f:
            json.dump(scenes, f, indent=4)

I. PLAYING PILGRIMS.
II. A MERRY CHRISTMAS.
III. THE LAURENCE BOY.
IV. BURDENS.
V. BEING NEIGHBORLY.
VI. BETH FINDS THE PALACE BEAUTIFUL.
VII. AMY'S VALLEY OF HUMILIATION.
VIII. JO MEETS APOLLYON.
IX. MEG GOES TO VANITY FAIR.
X. THE P. C. AND P. O.
XI. EXPERIMENTS.
XII. CAMP LAURENCE.
XIII. CASTLES IN THE AIR.
XIV. SECRETS.
XV. A TELEGRAM.
XVI. LETTERS.
XVII. LITTLE FAITHFUL.
XVIII. DARK DAYS.
XIX. AMY'S WILL.
XX. CONFIDENTIAL.
XXI. LAURIE MAKES MISCHIEF, AND JO MAKES PEACE.
XXII. PLEASANT MEADOWS.
XXIII. AUNT MARCH SETTLES THE QUESTION.
XXIV. GOSSIP.
XXV. THE FIRST WEDDING.
XXVI. ARTISTIC ATTEMPTS.
XXVII. LITERARY LESSONS.
XXVIII. DOMESTIC EXPERIENCES.
XXIX. CALLS.
XXX. CONSEQUENCES.
XXXI. OUR FOREIGN CORRESPONDENT.
XXXII. TENDER TROUBLES.
XXXIII. JO'S JOURNAL.
XXXIV. A FRIEND.
XXXV. HEARTACHE.
XXXVI. BETH'S SECRET.
XXXVII. NEW IMPRESSIONS.
XXXVIII. ON THE SHELF.
XXXIX. LAZY LAURENCE.
XL. THE VALLEY OF THE SHADOW.
XLI. LEARNING TO FORGET.
XLII. ALL ALONE.
XLIII. SURPRISES.
XLIV. MY LO

In [5114]:
# combine all chapters into one json file
if not scene_details_exist:
    all_scenes = []
    for i, marker in enumerate(markers):
        with open(f"json/{story_name}/{marker}.json") as f:
            scenes = json.load(f)
        all_scenes += scenes

    with open(f"json/{story_name}/all.json", "w") as f:
        json.dump(all_scenes, f, indent=4)

### grouping duplicate characters / location names

In [5115]:
# collect all unique characters and locations in the story
all_characters = {}
all_locations = []

if not scene_details_exist:
    for i, marker in enumerate(markers):
        with open(f"json/{story_name}/{marker}.json") as f:
            scenes = json.load(f)
        for scene in scenes:
            for character in scene["characters"]:
                char = character["name"]
                if char not in all_characters:
                    all_characters[char] = character["importance_exp"]
            location = scene["location"]
            if location not in all_locations:
                all_locations.append(location)
else:
    # load from all.json
    with open(f"json/{story_name}/all.json") as f:
        scenes = json.load(f)
    for scene in scenes:
        for character in scene["characters"]:
            char = character["name"]
            if char not in all_characters:
                all_characters[char] = character["importance_exp"]
        location = scene["location"]
        if location not in all_locations:
            all_locations.append(location)

In [5116]:
len(all_characters), len(all_locations)

(167, 214)

In [5117]:
all_characters

{'Jo March': 'Very important character expressing discontent',
 'Meg March': 'Very important character reflecting on sacrifices',
 'Amy March': 'Very important character expressing desire',
 'Beth March': 'Very important character providing comfort',
 'Mrs. March': 'Very important character as the mother figure',
 'Marmee (Mrs. March)': 'Central figure in the music tradition',
 'Hannah': 'Hannah provides context and background.',
 'Hugo': 'Hugo is the main villain of the play.',
 'Hagar': "Hagar plays a crucial role in the plot's resolution.",
 'Zara': 'Zara is central to the love story and conflict.',
 'Roderigo': 'Roderigo is the love interest and protagonist.',
 'Don Pedro': "Don Pedro is the antagonist to Roderigo and Zara's love.",
 'Meg': "Meg provides insight into the family's dynamics.",
 'Jo': 'Jo expresses a desire for friendship and connection.',
 'Beth': 'Beth shows affection and care for her family.',
 'Father': 'The father is mentioned but not directly quoted.',
 'Mrs. Ga

In [5118]:
all_locations

['Living Room',
 'March family home',
 'Home',
 'Christmas morning',
 'The March household',
 'Christmas night performance',
 'Christmas Gathering',
 'Garret stairs',
 'The parlor',
 "Mrs. Gardiner's house",
 'Curtained recess',
 'A party venue',
 'Home after the party',
 "Aunt March's house",
 "Beth's home",
 "Amy's School",
 'Sewing Room',
 'Snowy afternoon hall',
 'March family garden',
 "Jo's neighborhood",
 "Laurie's room",
 "Laurie's little parlor",
 "Jo's living room",
 "Laurie's Library",
 "Laurence's living room",
 "Mr. Laurence's home",
 'Big house on the hedge',
 "Mr. Laurence's House",
 'Old parlor',
 "Mr. Laurence's house",
 'drawing-room',
 'Great drawing-room',
 'March household',
 "Beth's home parlor",
 "Laurence's house",
 "Girls' Room",
 'On the road',
 "Amy's room",
 'School Classroom',
 'school classroom',
 'Classroom',
 'School platform',
 "Girls' bedroom",
 "Jo's home",
 'On the river',
 "Jo's bedroom",
 "Meg's room",
 "Moffat's house",
 'small party',
 'The garde

In [5119]:
# # create csv file for characters
# import csv

# with open(f"csv/{story_name}_characters.csv", "w", newline="") as f:
#     writer = csv.writer(f)
#     writer.writerow(["Character", "Role"])
#     for char, role in all_characters.items():
#         writer.writerow([char, role])

In [5120]:
# # create csv file for locations
# with open(f"csv/{story_name}_locations.csv", "w", newline="") as f:
#     writer = csv.writer(f)
#     writer.writerow(["Location"])
#     for location in all_locations:
#         writer.writerow([location])

In [5121]:
# Assuming your JSON file is stored at 'all.json'
with open(f"json/{story_name}/all.json") as f:
    json_data = json.load(f)

prompt = """
You are given a list of characters with some duplicates or alternate names.
Your task is to group all the alternate names for each unique character under a single full name, using the descriptions as guidance.
Don't keep characters like "Narrator" unless they are never referred to by another name.
Separate characters that are related to others (e.g., "Bob" and "Ms. Doe (Bob's mom)" and "Bob's sister") into separate entries.

First create a new character list by removing all duplicates and alternate names and only keeping the full name for each character.

Then, output a JSON dictionary where the key is each character in your new list and the value is a list of all possible alternate names. 
All names should be taken directly from the original character list and should only appear once in the output.

For example:
{{
    "Bob Smith": ["Bob", "Bobby", "Robert"],
    "Jane Doe": ["Jane", "Ms. Doe", "Bob's mother"]
}}
"""

theme_prompt = """
You are given a list of themes with some duplicates or alternate names.
Your task is to group all the alternate names for each unique theme under a single name, using the descriptions as guidance.
Your goal is to have as few themes as possible while still capturing all the different ways the themes are described.

First create a new theme list by removing all duplicates and alternate names and only keeping the best description for each theme.

Then, output a JSON dictionary where the key is each theme in your new list and the value is a list of all possible alternate names. 
All names should be taken directly from the original theme list and should only appear once in the output.

For example:
{{
    "Memory": ["Remembering the past", "Recollection", "Nostalgia"],
    "Friendship and Loyalty": ["Friendship and Companionship", "Making Friends", "Bonds of Friendship", "Loyalty and Friendship"]
}}
"""

location_prompt = """
You are given a list of locations with some duplicates or alternate names.
Your task is to group all the alternate or similiar names for each unique location under a single location name.

First create a new location list by removing all duplicates and alternate names and only keeping the full name for each location.
If one location is inside another or nearby to it, only keep the larger location (e.g., "Bob's bedroom", "Bob's garden", and "Bob Smith's castle" can be listed under "Bob's house").
Then, output a JSON dictionary where the key is each location in your new list and the value is a list of all possible alternate location names.
All locations should be taken directly from the original location list and should only appear once in the output dict.

For example:
{{
    "Los Angeles": ["LA", "City of Angels", "Cafe in Los Angeles"],
    "Bob's house": ["Bob's bedroom", "Bob Smith's garden", "Bob's castle"]
}}
"""

if analysis_type == "theme":
    prompt = theme_prompt

# if using llm2 (claude)
prompt += "Just output the JSON dictionary as the final result without any additional information."
location_prompt += "Just output the JSON dictionary as the final result without any additional information."

prompt_template = ChatPromptTemplate.from_messages([("system", prompt), ("human", "{input}")])
location_prompt_template = ChatPromptTemplate.from_messages([("system", location_prompt), ("human", "{input}")])

In [5122]:
# Create the LLM chain with the prompt template and model

# Run the chain by passing the characters and JSON data as inputs
if analysis_type == "theme":
    final_prompt = prompt_template.invoke(f"theme list: {all_characters}")
else:
    final_prompt = prompt_template.invoke(f"character list: {all_characters}")
# character_llm = llm.with_structured_output(None, method="json_mode") if model_type == "gpt" else llm
character_llm = llm2
# character_llm = llm
output = character_llm.invoke(final_prompt)


In [5123]:
output

AIMessage(content='{\n    "Jo March": ["Jo", "Josephine March", "Miss March", "Jo Bhaer"],\n    "Meg March": ["Meg", "Margaret March", "Margaret", "Margaret (Meg)", "Meg Brooke", "Mrs. Brooke"],\n    "Amy March": ["Amy", "Amy Curtis March", "Amy Laurence"],\n    "Beth March": ["Beth", "Elizabeth March"],\n    "Mrs. March": ["Marmee", "Marmee March", "Mother", "Mother March"],\n    "Hannah": ["Hannah Mullet"],\n    "Hugo": [],\n    "Hagar": [],\n    "Zara": [],\n    "Roderigo": [],\n    "Don Pedro": [],\n    "Father": ["Mr. March", "Father March", "Papa"],\n    "Mrs. Gardiner": [],\n    "Theodore Laurence": ["Laurie", "Laurence boy", "Laurie Laurence", "Laurie (Theodore Laurence)", "Theodore \'Laurie\' Laurence"],\n    "Aunt March": [],\n    "Mr. Laurence": ["James Laurence"],\n    "The girl": [],\n    "Miss Snow": [],\n    "Mr. Davis": [],\n    "Jenny": [],\n    "Meg Moffat": [],\n    "Annie Moffat": ["Annie"],\n    "Mrs. Moffat": [],\n    "Clara": [],\n    "Mr. Pickwick": [],\n    "Mr

In [5124]:
# if model_type == "claude":
output_formatted = output.content
# if output stop_reason = "max_tokens",cut off at last ']' character and add '}'
if output_formatted[-1] != "}":
    last_bracket_ind = output_formatted.rfind("]")
    output_formatted = output_formatted[:last_bracket_ind + 1] + "}"
    # print(output_formatted)
# turn into json
output = json.loads(output_formatted)

In [5125]:
output

{'Jo March': ['Jo', 'Josephine March', 'Miss March', 'Jo Bhaer'],
 'Meg March': ['Meg',
  'Margaret March',
  'Margaret',
  'Margaret (Meg)',
  'Meg Brooke',
  'Mrs. Brooke'],
 'Amy March': ['Amy', 'Amy Curtis March', 'Amy Laurence'],
 'Beth March': ['Beth', 'Elizabeth March'],
 'Mrs. March': ['Marmee', 'Marmee March', 'Mother', 'Mother March'],
 'Hannah': ['Hannah Mullet'],
 'Hugo': [],
 'Hagar': [],
 'Zara': [],
 'Roderigo': [],
 'Don Pedro': [],
 'Father': ['Mr. March', 'Father March', 'Papa'],
 'Mrs. Gardiner': [],
 'Theodore Laurence': ['Laurie',
  'Laurence boy',
  'Laurie Laurence',
  'Laurie (Theodore Laurence)',
  "Theodore 'Laurie' Laurence"],
 'Aunt March': [],
 'Mr. Laurence': ['James Laurence'],
 'The girl': [],
 'Miss Snow': [],
 'Mr. Davis': [],
 'Jenny': [],
 'Meg Moffat': [],
 'Annie Moffat': ['Annie'],
 'Mrs. Moffat': [],
 'Clara': [],
 'Mr. Pickwick': [],
 'Mr. Snodgrass': ['Snodgrass'],
 'Mr. Tupman': [],
 'Mr. Winkle': ['N. Winkle'],
 'Count de Adelon': [],
 'Viola

In [5126]:
# Create the LLM chain with the prompt template and model
if not scene_details_exist:
    # Run the chain by passing the characters and JSON data as inputs
    final_location_prompt = location_prompt_template.invoke(f"location list: {all_locations}")
    # location_llm = llm.with_structured_output(None, method="json_mode") if model_type == "gpt" else llm
    location_llm = llm2
    # location_llm = llm
    location_output = location_llm.invoke(final_location_prompt)

In [5127]:
if not scene_details_exist:
    print(location_output)

content='{\n    "March family home": ["March family home", "Home", "The March household", "March household", "March Family Home", "the March home", "The March family home", "Mr. March\'s home", "Jo\'s family home", "the March household", "Mrs. March\'s home"],\n    "Living Room": ["Living Room", "The parlor", "Parlor", "Drawing-room", "Great drawing-room", "Old parlor", "drawing-room", "the parlor"],\n    "Christmas Gathering": ["Christmas morning", "Christmas night performance", "Christmas Gathering", "Christmas dinner table", "Christmas party"],\n    "Garret": ["Garret stairs", "Big Garret", "Jo\'s garret"],\n    "Mrs. Gardiner\'s house": ["Mrs. Gardiner\'s house"],\n    "Curtained recess": ["Curtained recess", "Behind the curtain"],\n    "Party venue": ["A party venue", "small party"],\n    "Aunt March\'s house": ["Aunt March\'s house", "Aunt March\'s House"],\n    "Beth\'s home": ["Beth\'s home", "Beth\'s home parlor"],\n    "Amy\'s School": ["Amy\'s School", "School Classroom", "s

In [5128]:
if not scene_details_exist:
    location_output_formatted = location_output.content
    # turn into json
    location_output = json.loads(location_output_formatted)

In [5129]:
if not scene_details_exist: 
    print(location_output)

{'March family home': ['March family home', 'Home', 'The March household', 'March household', 'March Family Home', 'the March home', 'The March family home', "Mr. March's home", "Jo's family home", 'the March household', "Mrs. March's home"], 'Living Room': ['Living Room', 'The parlor', 'Parlor', 'Drawing-room', 'Great drawing-room', 'Old parlor', 'drawing-room', 'the parlor'], 'Christmas Gathering': ['Christmas morning', 'Christmas night performance', 'Christmas Gathering', 'Christmas dinner table', 'Christmas party'], 'Garret': ['Garret stairs', 'Big Garret', "Jo's garret"], "Mrs. Gardiner's house": ["Mrs. Gardiner's house"], 'Curtained recess': ['Curtained recess', 'Behind the curtain'], 'Party venue': ['A party venue', 'small party'], "Aunt March's house": ["Aunt March's house", "Aunt March's House"], "Beth's home": ["Beth's home", "Beth's home parlor"], "Amy's School": ["Amy's School", 'School Classroom', 'school classroom', 'Classroom', 'School platform'], 'Sewing Room': ['Sewing

In [5130]:
# Function to find the correct key based on a search string, ensuring whole-word matches
def find_key_from_alias(text, character_dict):
    for key, aliases in character_dict.items():
        for alias in sorted(aliases, key=len, reverse=True):
            # Match the alias as a whole word, and ensure no partial matches within other words
            pattern = rf'\b{re.escape(alias)}\b(?!\'s)'  # (?!\'s) ensures no match for possessives like Mitsuha's
            if re.search(pattern, text):
                return key  # Return the key if the alias is found as a whole word
    return text  # Return the original text if no match is found

In [5131]:
# Load the scenes
with open(f"json/{story_name}/all.json") as f:
    scenes = json.load(f)

# Capitalize output and location output
capitalized_output = {string.capwords(key): [string.capwords(name) for name in value] for key, value in output.items()}

if not scene_details_exist:
    capitalized_location_output = {string.capwords(key): [string.capwords(name) for name in value] for key, value in location_output.items()}

# Process each scene
for scene in scenes:
    # Process characters
    for character in scene["characters"]:
        char = string.capwords(character["name"])
        found_char = False

        # Check if the character's name is a key in the dictionary
        if char in capitalized_output:
            found_char = True
            character["name"] = char  # Capitalize the character name
        
        # If not found in the dictionary, check the aliases
        if not found_char:
            # Find the right key for the alias (if exists)
            matched_key = find_key_from_alias(char, capitalized_output)
            if matched_key != char:
                character["name"] = matched_key
                found_char = True
        if not found_char:
            character["name"] = char

    if not scene_details_exist:
        # Process location
        loc = string.capwords(scene["location"])
        found_loc = False

        # Check if the location is a key in the location dictionary
        if loc in capitalized_location_output:
            found_loc = True
            scene["location"] = loc  # Capitalize the location

        # If not found in the dictionary, check the aliases
        if not found_loc:
            matched_key = find_key_from_alias(loc, capitalized_location_output)
            if matched_key != loc:
                scene["location"] = matched_key
                found_loc = True
        if not found_loc:
            scene["location"] = loc

# Save the updated scenes
with open(f"json/{story_name}/all.json", "w") as f:
    json.dump(scenes, f, indent=4)

In [5132]:
# Remove duplicate characters in a scene and rank characters
# Also rename duplicate scene names

# Load the scenes
with open(f"json/{story_name}/all.json") as f:
    scenes = json.load(f)

scene_name_counts = {}

for j, scene in enumerate(scenes):
    scene_name = scene["title"]
    if scene_name in scene_name_counts:
        # update scene name if it is a duplicate
        scene_name_counts[scene_name] += 1
        scene["title"] = f"{scene_name} {scene_name_counts[scene_name]}"
    else:
        scene_name_counts[scene_name] = 1

    # remove duplicate characters
    unique_characters = []
    new_character_info = []
    for character in scene["characters"]:
        char_name = character["name"]
        if char_name not in unique_characters:
            unique_characters.append(char_name)
            new_character_info.append(character)
    scene["characters"] = new_character_info

    # now extract character importances and rank
    character_importances = []
    for k, character in enumerate(scene["characters"]):
        character_importances.append((k, character["importance"]))
    # sort character importances
    sorted_character_importances = sorted(character_importances, key=lambda x: x[1], reverse=True)
    # add importance_rank to each character
    for k, (l, _) in enumerate(sorted_character_importances):
        scenes[j]["characters"][l]["importance_rank"] = k+1

# Save the updated scenes
with open(f"json/{story_name}/all.json", "w") as f:
    json.dump(scenes, f, indent=4)

In [5133]:
# now list all characters and locations again
# collect all unique characters and locations for each chapter
all_characters_new = {}
all_locations_new = []
with open(f"json/{story_name}/all.json") as f:
    scenes = json.load(f)
for scene in scenes:
    for character in scene["characters"]:
        char = character["name"]
        if char not in all_characters_new:
            all_characters_new[char] = 1
        else:
            all_characters_new[char] += 1
    location = scene["location"]
    if location not in all_locations_new:
        all_locations_new.append(location)

In [5134]:
len(all_characters_new), len(all_locations_new)

(112, 123)

In [5135]:
all_characters_new 

{'Jo March': 210,
 'Meg March': 134,
 'Amy March': 129,
 'Beth March': 96,
 'Mrs. March': 62,
 'Hannah': 26,
 'Hugo': 1,
 'Hagar': 1,
 'Zara': 1,
 'Roderigo': 1,
 'Don Pedro': 1,
 'Father': 22,
 'Mrs. Gardiner': 1,
 'Theodore Laurence': 102,
 'Aunt March': 13,
 'Mr. Laurence': 22,
 'The Girl': 1,
 'Miss Snow': 1,
 'Mr. Davis': 3,
 'Jenny': 1,
 'Meg Moffat': 1,
 'Annie Moffat': 2,
 'Mrs. Moffat': 2,
 'Clara': 1,
 'Mr. Pickwick': 3,
 'Mr. Snodgrass': 2,
 'Mr. Tupman': 1,
 'Mr. Winkle': 2,
 'Count De Adelon': 1,
 'Viola': 1,
 'Ferdinand Devereux': 1,
 'The Troubadour': 1,
 'The Lady': 1,
 'The Farmer': 1,
 'The Grocer-man': 1,
 'The Little Girl': 1,
 'The March Family': 1,
 'Mrs. Snowball Pat Paw': 1,
 'The Community': 1,
 'Sympathizing Friend': 1,
 'Snowball': 1,
 'New Cat': 1,
 'Miss Oranthy Bluggage': 1,
 'Hannah Brown': 1,
 'Mrs. Beth Bouncer': 1,
 'Sam Weller': 1,
 'The Old Gentleman': 3,
 'Miss Crocker': 3,
 'Kate': 5,
 'Ned': 3,
 'Fred Vaughn': 6,
 'Mr. Brooke': 36,
 'Frank': 2,
 '

In [5136]:
all_locations_new

['Living Room',
 'March Family Home',
 'Christmas Gathering',
 'Garret',
 "Mrs. Gardiner's House",
 'Curtained Recess',
 'Party Venue',
 "Aunt March's House",
 "Beth's Home",
 "Amy's School",
 'Sewing Room',
 'Snowy Afternoon Hall',
 'March Family Garden',
 "Jo's Neighborhood",
 "Laurie's House",
 "Girls' Room",
 'On The Road',
 "Amy's Room",
 "Jo's Home",
 'On The River',
 "Moffat's House",
 'Pickwick Hall',
 "Count De Adelon's Halls",
 "Farmer's Garden",
 'Club Basement',
 'Community Gathering',
 'Weekly Report Meeting',
 'P. C. Club Meeting',
 'The P. O. Post-office',
 "Meg's Home",
 'The Lawn',
 'Camp Laurence',
 'Outdoor Picnic Area',
 'Under The Trees',
 "Laurie's Backyard Hammock",
 'The Woods',
 'Under The Birches',
 'Delectable Mountain',
 'Stairs And Gymnasium',
 'At The Gate',
 'Outside, Parting Ways',
 'Old House',
 "Mother's Closet",
 "Beth's Sickroom",
 "Jo And Meg's Home",
 "Amy's Will",
 'The Chapel',
 "Mrs. March's Room",
 "Mrs. March's Parlor",
 "Mr. Laurence's Study"

### double check character quotes

In [5137]:
def is_direct_quote(quote, scene_text):
    scene_text = scene_text.replace("\n", " ").replace("_", "").lower()
    scene_text = re.sub(r'[^\w\s]', '', scene_text) # remove punctuation

    quote = quote.replace("_", "").lower()
    quote = re.sub(r'[^\w\s]', '', quote) # remove punctuation

    return quote in scene_text

In [5138]:
wrong_quotes = []

# load in scenes
with open(f"json/{story_name}/all.json") as f:
    scenes = json.load(f)

    for scene in scenes:
        scene_text = scene["text"]
        scene_name = scene["title"]
        scene_chapter = scene["chapter"]
        for character in scene["characters"]:
            char_name = character["name"]
            quote = character["quote"]
            emotion = character["emotion"]
            if "No quote available" not in character["quote"] and not is_direct_quote(quote, scene_text):
                wrong_quotes.append({"scene": scene_name, "chapter": scene_chapter, "character": char_name, "emotion": emotion, "quote": quote})

In [5139]:
len(wrong_quotes), wrong_quotes

(22,
 [{'scene': 'The Christmas Play',
   'chapter': 'II. A MERRY CHRISTMAS.',
   'character': 'Beth March',
   'emotion': 'loving and gentle',
   'quote': 'Beth nestled up to her, and whispered softly, "I wish I could send my roses to him."'},
  {'scene': "Jo's Ambition",
   'chapter': 'IV. BURDENS.',
   'character': 'Jo March',
   'emotion': 'Frustrated yet content',
   'quote': "I couldn't bear it any longer;"},
  {'scene': "Amy's Trials",
   'chapter': 'IV. BURDENS.',
   'character': 'Amy March',
   'emotion': 'Self-conscious and frustrated',
   'quote': "My only comfort is, that mother don't take tucks in my dresses whenever I'm naughty, as Maria Parks' mother does."},
  {'scene': "Laurie's Visit",
   'chapter': "VII. AMY'S VALLEY OF HUMILIATION.",
   'character': 'Amy March',
   'emotion': 'thoughtful',
   'quote': '"Is Laurie an accomplished boy?" asked Amy.'},
  {'scene': 'The Public Bereavement',
   'chapter': 'X. THE P. C. AND P. O.',
   'character': 'Mrs. Snowball Pat Paw',


In [5140]:
# Pydantic
class CharacterExplanation(BaseModel):
    """Get information about a character's quotes in the story."""
    number: int = Field(description="Quote number")
    character: str = Field(description="Character name")
    exp: str = Field(description="Brief, 1-sentence explanation of why the character is feeling the emotion based on the quote. Include relevant context from the quote but do not directly refer to it.")

class CharacterExplanationList(BaseModel):
    """Get overall information about the story."""
    characters: list[CharacterExplanation] = Field(description="List of character emotion explanations")

class ThemeExplanation(BaseModel):
    """Get information about a theme's quotes in the story."""
    number: int = Field(description="Quote number")
    theme: str = Field(description="Theme name")
    exp: str = Field(description="Brief, 1-sentence explanation of how the theme reflects to the emotion. Include relevant context from the quote but do not directly refer to it.")

class ThemeExplanationList(BaseModel):
    """Get overall information about the story."""
    themes: list[ThemeExplanation] = Field(description="List of theme emotion explanations")

char_exp_llm = llm.with_structured_output(CharacterExplanationList)
theme_exp_llm = llm.with_structured_output(ThemeExplanationList)

In [5141]:
# async function to get character explanations
async def get_character_explanations_async(quotes):
    print(f"Running one batch...")

    quotes_formatted = [{"number": i, "character": quote["character"], "emotion": quote["emotion"], "quote": quote["quote"]} for i, quote in enumerate(quotes)]

    prompt = f"""
             character quotes: {quotes_formatted}

             your output should contain exactly the same {len(quotes_formatted)} characters as the input.
             """
    
    theme_prompt = f"""
                theme quotes: {quotes_formatted}

                your output should contain exactly the same {len(quotes_formatted)} themes as the input.
                """
    
    character_explanations = await char_exp_llm.ainvoke(prompt) if analysis_type == "character" else await theme_exp_llm.ainvoke(theme_prompt)
    explanations = character_explanations.characters if analysis_type == "character" else character_explanations.themes

    for explanation in explanations:
        num = explanation.number
        exp = explanation.exp
        char = explanation.character if analysis_type == "character" else explanation.theme
        if (char != quotes[num]["character"]):
            print("Error: character name does not match")
        quotes[num]["exp"] = exp
    
    return quotes

In [5142]:
# run async function in parallel
max_quotes_per_batch = 20
tasks = [semaphore_wrapper(get_character_explanations_async, wrong_quotes[i:i+max_quotes_per_batch]) for i in range(0, len(wrong_quotes), max_quotes_per_batch)]
final_quotes = await asyncio.gather(*tasks)
print("Done")

Running one batch...
Running one batch...
Done


In [5143]:
final_quotes

[[{'scene': 'The Christmas Play',
   'chapter': 'II. A MERRY CHRISTMAS.',
   'character': 'Beth March',
   'emotion': 'loving and gentle',
   'quote': 'Beth nestled up to her, and whispered softly, "I wish I could send my roses to him."',
   'exp': 'Beth expresses her loving nature and gentle spirit by wishing to send roses to someone she cares about.'},
  {'scene': "Jo's Ambition",
   'chapter': 'IV. BURDENS.',
   'character': 'Jo March',
   'emotion': 'Frustrated yet content',
   'quote': "I couldn't bear it any longer;",
   'exp': "Jo's frustration stems from her inability to endure a situation any longer, yet she finds a sense of contentment in her resolve."},
  {'scene': "Amy's Trials",
   'chapter': 'IV. BURDENS.',
   'character': 'Amy March',
   'emotion': 'Self-conscious and frustrated',
   'quote': "My only comfort is, that mother don't take tucks in my dresses whenever I'm naughty, as Maria Parks' mother does.",
   'exp': 'Amy feels self-conscious and frustrated about her app

In [5144]:
# replace wrong quotes with explanations
for i, quotes in enumerate(final_quotes):
    for quote in quotes:
        s = quote["scene"]
        chapter = quote["chapter"]
        char = quote["character"]

        # find scene
        for scene in scenes:
            scene_name = scene["title"]
            if scene_name == s and scene["chapter"] == chapter:
                for character in scene["characters"]:
                    if character["name"] == char:
                        character["quote"] = "No quote available"
                        character["fake_quote"] = quote["exp"]
                        print("Fixed quote for", char, "in", scene_name)
                        break

Fixed quote for Beth March in The Christmas Play
Fixed quote for Jo March in Jo's Ambition
Fixed quote for Amy March in Amy's Trials
Fixed quote for Amy March in Laurie's Visit
Fixed quote for Mrs. Snowball Pat Paw in The Public Bereavement
Fixed quote for The Community in The Public Bereavement
Fixed quote for Meg March in Morning After Vacation Starts
Fixed quote for Jo March in Morning After Vacation Starts
Fixed quote for Mrs. March in Jo's Letters
Fixed quote for Mr. Brooke in Preparing for Departure
Fixed quote for Meg March in Letters to Mother
Fixed quote for Meg March in Beth's Illness
Fixed quote for Father in Christmas Dinner
Fixed quote for Mrs. March in Christmas Dinner
Fixed quote for Amy March in Reflections and Music
Fixed quote for Meg March in Sisters' Reactions
Fixed quote for Meg March in Aftermath of the Dinner
Fixed quote for Amy March in Aunt Carrol's Invitation
Fixed quote for Fred Vaughn in Journey to Heidelberg
Fixed quote for Meg March in Meg's Growing Loneli

In [5145]:
# replace in json
with open(f"json/{story_name}/all.json", "w") as f:
    json.dump(scenes, f, indent=4)

## produce chapter summaries

In [5146]:
# Pydantic
class ChapterSummary(BaseModel):
    """Summarize a chapter based on scenes."""
    summary: str = Field(description="A brief, 1-line summary of the chapter")

chapter_llm = llm.with_structured_output(ChapterSummary)

In [5147]:
class CharacterLink(BaseModel):
    """Summarize the interaction between two characters in this chapter."""
    char1: str = Field(description="Name of the first character")
    char2: str = Field(description="Name of the second character")
    interaction: str = Field(description="Brief description of the interaction between the two characters")

class CharacterLinks(BaseModel):
    """Summarize all interactions between characters in a chapter."""
    links: list[CharacterLink] = Field(description="List of interactions between pairs of characters in the scene")

class ThemeLink(BaseModel):
    """Summarize the interaction between two themes in this chapter."""
    char1: str = Field(description="Name of the first theme")
    char2: str = Field(description="Name of the second theme")
    interaction: str = Field(description="Brief description of the interaction between the two themes")

class ThemeLinks(BaseModel):
    """Summarize all interactions between themes in a chapter."""
    links: list[ThemeLink] = Field(description="List of interactions between pairs of themes in the scene")

character_links_llm = llm.with_structured_output(CharacterLinks)
theme_links_llm = llm.with_structured_output(ThemeLinks)

In [5148]:
# remove any markers without any scenes
if not scene_details_exist:
    markers_to_remove = []
    for i, marker in enumerate(markers):
        with open(f"json/{story_name}/{marker}.json") as f:
            scenes = json.load(f)
        if len(scenes) == 0:
            markers_to_remove.append(marker)

    for marker in markers_to_remove:
        markers.remove(marker)

In [5149]:
async def read_json_async(file_path):
    """Asynchronously read a JSON file."""
    async with aiofiles.open(file_path, mode="r") as f:
        data = await f.read()
        return json.loads(data)
    
async def read_text_async(file_path):
    """Asynchronously read a text file."""
    async with aiofiles.open(file_path, mode="r") as f:
        return await f.read()
    
async def process_chapter_summary_async(og_story_name, marker, scenes):
    """Process the summary for a single chapter."""
    chapter_file = f"chapters/{og_story_name}/{marker}.txt"

    chapter_scenes = [scene for scene in scenes if scene["chapter"] == marker]

    # Read chapter text file asynchronously
    chapter_text = await read_text_async(chapter_file)

    # Create scene summaries string
    scene_summaries = " ".join([scene["summary"] for scene in chapter_scenes])

    # Get summary from LLM
    summary = await chapter_llm.ainvoke(scene_summaries)
    print(f"Chapter {marker} summary generated.")

    # Create character links 
    links = []
    for scene in chapter_scenes:
        characters = scene["characters"]
        for j in range(len(characters)):
            char1 = characters[j]
            char1_name = char1["name"]
            for k in range(j+1, len(characters)):
                char2 = characters[k]
                char2_name = char2["name"]
                
                # check if link already exists
                existing_link = next((link for link in links if (link["source"] == char1_name and link["target"] == char2_name)
                                        or (link["source"] == char2_name and link["target"] == char1_name)), None)
                if existing_link:
                    # increment value if link already exists
                    existing_link["value"] += 1
                else:
                    # add new link
                    links.append({"source": char1_name, "target": char2_name, "value": 1})
    
    # Create character links summary
    character_links = [{"char1": link["source"], "char2": link["target"]} for link in links]
    link_prompt = f"""
                    Character pairs: {character_links}
                    Scene summaries: {scene_summaries}

                    Based on the scene summaries above, summarize how each pair of characters interacts in this chapter.
                    Your output should contain exactly the same {len(character_links)} pairs as in the original list.
                    """
    theme_link_prompt = f"""
                    Theme pairs: {character_links}
                    Scene summaries: {scene_summaries}

                    Based on the scene summaries above, summarize how each pair of themes interact in this chapter.
                    Your output should contain exactly the same {len(character_links)} pairs as in the original list.
                    """

    # Get character links from LLM
    character_links_summary = await character_links_llm.ainvoke(link_prompt) if analysis_type == "character" else await theme_links_llm.ainvoke(theme_link_prompt)
    character_links_list = character_links_summary.links

    print(f"Character links for Chapter {marker} generated.")

    # Add interactions to character links using the LLM output
    for llm_link in character_links_list:
        char1_name = llm_link.char1
        char2_name = llm_link.char2
        for link in links:
            if (link["source"] == char1_name and link["target"] == char2_name) or (link["source"] == char2_name and link["target"] == char1_name):
                link["interaction"] = llm_link.interaction

    chapter_summary = {
        "chapter": marker,
        "scenes": len(chapter_scenes),
        "length": len(chapter_text),
        "num_lines": len(chapter_text.split("\n")),
        "summary": summary.summary,
        "importance": [],
        "conflict": [],
        "locations": {},
        "characters": {},
        "links": links
    }

    return chapter_summary

async def generate_chapter_summaries_async(story_name, og_story_name, markers):
    """Generate summaries for all chapters in parallel."""
    # Read all.json file asynchronously
    scenes = await read_json_async(f"json/{story_name}/all.json")

    # Processs chapters in parallel
    chapter_summaries = await asyncio.gather(*[process_chapter_summary_async(og_story_name, marker, scenes) for marker in markers])
    print("All chapter summaries generated.")

    # Aggregate character and location information
    for chapter_summary in chapter_summaries:
        chapter = chapter_summary["chapter"]

        chapter_scenes = [scene for scene in scenes if scene["chapter"] == chapter]
    
        for scene in chapter_scenes:
            # add location to chapter summary
            location = scene["location"]
            chapter_summary_locations = chapter_summary["locations"]
            if location not in chapter_summary_locations:
                chapter_summary_locations[location] = 1
            else:
                chapter_summary_locations[location] += 1
            # add importance to chapter summary
            importance = scene["importance"]
            chapter_summary["importance"].append(importance)
            # add conflict to chapter summary
            conflict = scene["conflict"]
            chapter_summary["conflict"].append(conflict)
            # add characters to chapter summary
            characters = scene["characters"]
            for character in characters:
                char = character["name"]
                chapter_summary_characters = chapter_summary["characters"]
                if char not in chapter_summary_characters:
                    chapter_summary_characters[char] = 1
                else:
                    chapter_summary_characters[char] += 1

    # now average importance and conflict for each chapter
    for chapter_summary in chapter_summaries:
        if len(chapter_summary["importance"]) == 0:
            print(f"Chapter {chapter_summary['chapter']} has no scenes.")
            continue
        chapter_summary["importance"] = round(sum(chapter_summary["importance"]) / len(chapter_summary["importance"]), 2)
        chapter_summary["conflict"] = round(sum(chapter_summary["conflict"]) / len(chapter_summary["conflict"]), 2)

    print("Importances", [chapter_summary["importance"] for chapter_summary in chapter_summaries])
    print("Conflicts", [chapter_summary["conflict"] for chapter_summary in chapter_summaries])

    # rank each chapter by importance and conflict
    importances = []
    conflicts = []
    for chapter_summary in chapter_summaries:
        importances.append((chapter_summary["chapter"], chapter_summary["importance"]))
        conflicts.append((chapter_summary["chapter"], chapter_summary["conflict"]))
        
    # sort importances
    sorted_importances = sorted(importances, key=lambda x: x[1], reverse=True)
    # add importance_rank to each chaptera
    for k, (j, _) in enumerate(sorted_importances):
        chapter_summaries[markers.index(j)]["importance_rank"] = k+1

    # sort conflicts
    sorted_conflicts = sorted(conflicts, key=lambda x: x[1], reverse=True)
    # add conflict_rank to each chapter
    for k, (j, _) in enumerate(sorted_conflicts):
        chapter_summaries[markers.index(j)]["conflict_rank"] = k+1

    return chapter_summaries

In [5150]:
async def process_chapter_summary_async_characters_only(chapter_summary, marker, scenes):
    """Process the summary for a single chapter."""

    chapter_scenes = [scene for scene in scenes if scene["chapter"] == marker]

    # Create scene summaries string
    scene_summaries = " ".join([scene["summary"] for scene in chapter_scenes])

    # Create character links 
    links = []
    for scene in chapter_scenes:
        characters = scene["characters"]
        for j in range(len(characters)):
            char1 = characters[j]
            char1_name = char1["name"]
            for k in range(j+1, len(characters)):
                char2 = characters[k]
                char2_name = char2["name"]
                
                # check if link already exists
                existing_link = next((link for link in links if (link["source"] == char1_name and link["target"] == char2_name)
                                        or (link["source"] == char2_name and link["target"] == char1_name)), None)
                if existing_link:
                    # increment value if link already exists
                    existing_link["value"] += 1
                else:
                    # add new link
                    links.append({"source": char1_name, "target": char2_name, "value": 1})
    
    # Create character links summary
    character_links = [{"char1": link["source"], "char2": link["target"]} for link in links]
    link_prompt = f"""
                    Character pairs: {character_links}
                    Scene summaries: {scene_summaries}

                    Based on the scene summaries above, summarize how each pair of characters interacts in this chapter.
                    Your output should contain exactly the same {len(character_links)} pairs as in the original list.
                    """
    theme_link_prompt = f"""
                    Theme pairs: {character_links}
                    Scene summaries: {scene_summaries}

                    Based on the scene summaries above, summarize how each pair of themes interact in this chapter.
                    Your output should contain exactly the same {len(character_links)} pairs as in the original list.
                    """

    # Get character links from LLM
    character_links_summary = await character_links_llm.ainvoke(link_prompt) if analysis_type == "character" else await theme_links_llm.ainvoke(theme_link_prompt)
    character_links_list = character_links_summary.links

    print(f"Character links for Chapter {marker} generated.")

    # Add interactions to character links using the LLM output
    for llm_link in character_links_list:
        char1_name = llm_link.char1
        char2_name = llm_link.char2
        for link in links:
            if (link["source"] == char1_name and link["target"] == char2_name) or (link["source"] == char2_name and link["target"] == char1_name):
                link["interaction"] = llm_link.interaction

    chapter_summary["links"] = links
    chapter_summary["characters"] = {}

    return chapter_summary

async def generate_chapter_summaries_async_characters_only(story_name, og_story_name, markers):
    """Generate summaries for all chapters in parallel."""
    print(f"Updating existing chapter information for {analysis_type} data...")
    # Read all.json file asynchronously
    scenes = await read_json_async(f"json/{story_name}/all.json")

    with open(f"json/{og_story_name}/final_data.json") as f:
        chapters_old = json.load(f)["chapters"]
    chapter_summaries = await asyncio.gather(*[process_chapter_summary_async_characters_only(chapter_summary, marker, scenes) for chapter_summary, marker in zip(chapters_old, markers)])

    # Aggregate character and location information
    for chapter_summary in chapter_summaries:
        chapter = chapter_summary["chapter"]

        chapter_scenes = [scene for scene in scenes if scene["chapter"] == chapter]
    
        for scene in chapter_scenes:
            # add characters to chapter summary
            characters = scene["characters"]
            for character in characters:
                char = character["name"]
                chapter_summary_characters = chapter_summary["characters"]
                if char not in chapter_summary_characters:
                    chapter_summary_characters[char] = 1
                else:
                    chapter_summary_characters[char] += 1

    return chapter_summaries

In [5151]:
# create summary json of each chapter
chapter_summaries = await generate_chapter_summaries_async(story_name, og_story_name, markers) if not scene_details_exist else await generate_chapter_summaries_async_characters_only(story_name, og_story_name, markers)

Chapter III. THE LAURENCE BOY. summary generated.
Chapter XXXII. TENDER TROUBLES. summary generated.
Chapter XXIII. AUNT MARCH SETTLES THE QUESTION. summary generated.
Chapter XXVI. ARTISTIC ATTEMPTS. summary generated.
Chapter XVI. LETTERS. summary generated.
Chapter I. PLAYING PILGRIMS. summary generated.
Chapter XLVII. HARVEST TIME. summary generated.
Chapter IV. BURDENS. summary generated.
Chapter XXXVIII. ON THE SHELF. summary generated.
Chapter XLVI. UNDER THE UMBRELLA. summary generated.
Chapter XLIV. MY LORD AND LADY. summary generated.
Chapter XII. CAMP LAURENCE. summary generated.
Chapter IX. MEG GOES TO VANITY FAIR. summary generated.
Chapter XXX. CONSEQUENCES. summary generated.
Chapter VI. BETH FINDS THE PALACE BEAUTIFUL. summary generated.
Chapter XXXIV. A FRIEND. summary generated.
Chapter VIII. JO MEETS APOLLYON. summary generated.
Chapter XXXVI. BETH'S SECRET. summary generated.
Chapter XLI. LEARNING TO FORGET. summary generated.
Chapter XLIII. SURPRISES. summary gener

In [5152]:
chapter_summaries

[{'chapter': 'I. PLAYING PILGRIMS.',
  'scenes': 5,
  'length': 26104,
  'num_lines': 368,
  'summary': "The sisters prioritize their mother's happiness over their own by choosing to buy her gifts, reflecting on their childhood and family traditions.",
  'importance': 0.82,
  'conflict': 0.22,
  'locations': {'Living Room': 2, 'March Family Home': 3},
  'characters': {'Jo March': 5,
   'Meg March': 5,
   'Amy March': 5,
   'Beth March': 5,
   'Mrs. March': 3},
  'links': [{'source': 'Jo March',
    'target': 'Meg March',
    'value': 5,
    'interaction': 'Jo and Meg express their disappointment about not receiving gifts for Christmas, discussing their feelings openly.'},
   {'source': 'Jo March',
    'target': 'Amy March',
    'value': 5,
    'interaction': 'Jo and Amy share their thoughts on the Christmas gifts, with Jo encouraging Amy to think of others.'},
   {'source': 'Jo March',
    'target': 'Beth March',
    'value': 5,
    'interaction': 'Jo comforts Beth as they talk about t

In [5153]:
# read in scenes from all_json
with open(f"json/{story_name}/all.json") as f:
    scenes = json.load(f)

In [5154]:
all_json = {}
all_json["title"] = story_info["title"]
all_json["type"] = story_info["type"]
if (all_json["type"] == "Movie"):
    all_json["director"] = story_info["director"]
else:
    all_json["author"] = story_info["author"]
all_json["year"] = story_info["year"]
if "url" in story_info:
    all_json["url"] = story_info["url"]
if "image" in story_info:
    all_json["image"] = story_info["image"]
all_json["num_chapters"] = len(chapters)
all_json["num_scenes"] = len(scenes)
all_json["num_characters"] = len(all_characters_new)
all_json["num_locations"] = len(all_locations_new)
all_json["chapters"] = chapter_summaries
all_json["scenes"] = scenes

## produce character summaries

In [5155]:
# Pydantic
class CharacterInfo(BaseModel):
    """Get information about a character in the story."""
    name: str = Field(description="Name of character.")
    quote: str = Field(description="Direct quote from the story that the character says or that describes them.")
    group: str = Field(description="Group that this character belongs to (e.g., main characters, upper class, talking animals). There should be at least 2 different groups of characters.")
    # color: str = Field(description="Unique aesthetic RGB color string that represents this character (e.g., rgb(118, 185, 71)). Every character should have a different color. Don't use white and make sure the color is visible against a white background.")
    # explanation: str = Field(description="Explanation why this color was chosen for this character.")

class LegendThemeInfo(BaseModel):
    """Get information about a theme in the story."""
    name: str = Field(description="Name of theme.")
    quote: str = Field(description="Direct quote from the story that describes this theme.")
    group: Optional[str] = Field(description="Group that this theme belongs to (e.g., Family, Social Themes, Emotions). There should be at least 2 different groups of themes. Make sure all groups are distinct and don't overlap with each other.")

class CharacterList(BaseModel):
    """Get overall information about the story."""
    # characterlist: list[CharacterInfo] = Field(description="List of character details. Make sure there is exactly one entry per character in the provided list and no additional characters are added. Choose a different color for each character.")
    characterlist: list[CharacterInfo] = Field(description="List of character details. Make sure there is exactly one entry per character in the provided list and no additional characters are added.")

class LegendThemeList(BaseModel):
     """Get overall information about the story."""
     themelist: list[LegendThemeInfo] = Field(description="List of theme details. Make sure there is exactly one entry per theme in the provided list and no additional themes are added.")

class ColorInfo(BaseModel):
    """Pick a color + explain why for each character"""
    name: str = Field(description="Name of character.")
    color: str = Field(description="Unique aesthetic RGB color string that represents this character (e.g., rgb(118, 185, 71)). Every character should have a different color. Don't use white and make sure the color is visible against a white background.")
    explanation: str = Field(description="Explanation why this color was chosen for this character.")

class ThemeColorInfo(BaseModel):
    """Pick a color + explain why for each theme"""
    name: str = Field(description="Name of theme.")
    color: str = Field(description="Unique aesthetic RGB color string that represents this theme (e.g., rgb(118, 185, 71)). Every theme should have a different color. Don't use white and make sure the color is visible against a white background.")
    explanation: str = Field(description="Explanation why this color was chosen for this theme.")

class ColorList(BaseModel):
    """List of color info for characters"""
    colorlist: list[ColorInfo] = Field(description="List of color details for each character. Make sure there is exactly one entry per character in the provided list and no additional characters are added. Choose a different color for each character.")

class ThemeColorList(BaseModel):
    """List of color info for themes"""
    colorlist: list[ThemeColorInfo] = Field(description="List of color details for each theme. Make sure there is exactly one entry per theme in the provided list and no additional themes are added. Choose a different color for each theme.")

if analysis_type == "theme":
    characterlist_llm = llm.with_structured_output(LegendThemeList)
    colorlist_llm = llm.with_structured_output(ThemeColorList)
else:
    characterlist_llm = llm.with_structured_output(CharacterList)
    colorlist_llm = llm.with_structured_output(ColorList)

In [5156]:
# sort all_characters_new by count, descending
sorted_characters = dict(sorted(all_characters_new.items(), key=lambda item: item[1], reverse=True))
# take top 20 characters
top_characters = dict(list(sorted_characters.items())[:20])
top_characters

{'Jo March': 210,
 'Meg March': 134,
 'Amy March': 129,
 'Theodore Laurence': 102,
 'Beth March': 96,
 'Mrs. March': 62,
 'Mr. Brooke': 36,
 'Hannah': 26,
 'Father': 22,
 'Mr. Laurence': 22,
 'Professor Bhaer': 22,
 'Aunt March': 13,
 'Demi March': 10,
 'Fred Vaughn': 6,
 'Kate': 5,
 'May Chester': 4,
 'Mr. Davis': 3,
 'Mr. Pickwick': 3,
 'The Old Gentleman': 3,
 'Miss Crocker': 3}

In [5157]:
character_arr = []
cur_arr = []
max_chars = 20

char_names = list(all_characters_new.keys())
# char_names = list(top_characters.keys())
for i, c in enumerate(char_names):
    cur_arr.append(c)
    if len(cur_arr) == max_chars or i == len(char_names) - 1:
        character_arr.append(cur_arr)
        cur_arr = []

for arr in character_arr:
    print(len(arr))


20
20
20
20
20
12


In [5158]:
async def process_character_chunk(i, arr):
    """Process a single chunk of character assignment asynchronously."""
    invalid_input = True
    while invalid_input:
        try:
            # filter scenes to include only those with relevant characters
            new_scenes = [scene for scene in scenes if any(char["name"] in arr for char in scene["characters"])]

             # take middle 126000 characters of new_scenes
            new_scenes_str = json.dumps(new_scenes)
            new_scenes_str = new_scenes_str[:126000]

            # construct prompt
            prompt = f"""characters: {arr}
                         story info: {new_scenes_str}

                         your output should contain exactly the same {len(arr)} characters as in the original list.

                         remember to only use direct quotes from the text and not paraphrase or change the text in any way.
                         """

            # invoke model
            res = await characterlist_llm.ainvoke(prompt)

            print(f"Chunk {i} processed.")
            return res

        except Exception as e:
            print("Exception thrown. Trying again.")
            print("Error:", e)
            await asyncio.sleep(1) # wait for 1 second before trying again

async def process_all_characters():
    """Process all characters in parallel."""
    tasks = [process_character_chunk(i, arr) for i, arr in enumerate(character_arr)]
    results = await asyncio.gather(*tasks)
    return results

In [5159]:
# Run the main function to process all characters
all_res = await process_all_characters()

Chunk 5 processed.
Chunk 4 processed.
Chunk 1 processed.
Chunk 2 processed.
Chunk 0 processed.
Chunk 3 processed.


In [5160]:
# combine characterlists into one
characters = []
for res in all_res:
    if analysis_type == "theme":
        print(len(res.themelist))
        characters += res.themelist
    else:
        print(len(res.characterlist))
        characters += res.characterlist

len(characters)

20
20
20
20
20
12


112

In [5161]:
character_names = [char.name for char in characters]
for name in character_names:
    if name not in char_names:
        print("not in orig list:", name)

need_to_add = []
for name in char_names:
    if name not in character_names:
        print("not in llm output:", name)
        need_to_add.append(name)

# find duplicate names
unique_names = []
duplicate_indices = []
for i, name in enumerate(character_names):
    if name not in unique_names:
        unique_names.append(name)
    else:
        print("duplicate:", name, i)
        duplicate_indices.append(i)

print("need_to_add", len(need_to_add))
print("duplicate_indices", len(duplicate_indices))


need_to_add 0
duplicate_indices 0


### group duplicate character groups

In [5162]:
# group similar groups

group_prompt = """
You are given a list of groups with some duplicates or alternate names.
Your task is to group all the alternate names for each unique group under a single name.
Your goal is to have as few groups as possible while still capturing all the unique groups.

First create a new group list by removing all duplicates and alternate names and only keeping the best description for each group.

Then, output a JSON dictionary where the key is each group in your new list and the value is a list of all possible alternate names. 
All names should be taken directly from the original group list and should only appear once in the output.

For example:
{{
    "Main Characters": ["Main Characters", "Protagonists", "Heroes"],
    "Elders": ["Elders", "Family Authorities"],
    "Exploration": ["Exploration", "Adventure", "Discovery"],
    "Relationships": ["Relationships", "Friendships"]
}}
"""

# if using llm2 (claude)
group_prompt += "Just output the JSON dictionary as the final result without any additional information."

group_prompt_template = ChatPromptTemplate.from_messages([("system", group_prompt), ("human", "{input}")])

In [5163]:
# get all unique groups
# groups = [c['group'] for c in characters]
groups = [c.group for c in characters]
unique_groups = list(set(groups))
len(unique_groups), unique_groups

(13,
 ['common folk',
  'Upper class characters',
  'Supporting characters',
  'main characters',
  'Romantic characters',
  'upper class',
  'supporting characters',
  'Supportive characters',
  'other characters',
  'Main characters',
  'community',
  'Social characters',
  'talking animals'])

In [5164]:
# Create the LLM chain with the prompt template and model

# Run the chain by passing the characters and JSON data as inputs
if analysis_type == "theme":
    final_prompt = group_prompt_template.invoke(f"list of groups for themes: {unique_groups}")
else:
    final_prompt = group_prompt_template.invoke(f"list of groups for characters: {unique_groups}")
group_llm = llm2
output = group_llm.invoke(final_prompt)

In [5165]:
output

AIMessage(content='{\n  "Common Folk": ["common folk", "community"],\n  "Upper Class Characters": ["Upper class characters", "upper class"],\n  "Supporting Characters": ["Supporting characters", "supporting characters", "Supportive characters"],\n  "Main Characters": ["main characters", "Main characters"],\n  "Romantic Characters": ["Romantic characters"],\n  "Other Characters": ["other characters", "Social characters"],\n  "Talking Animals": ["talking animals"]\n}', additional_kwargs={}, response_metadata={'id': 'msg_01NuHgEVcTQWK9p1AmNbBFEP', 'model': 'claude-3-5-sonnet-20240620', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 298, 'output_tokens': 106}}, id='run-5496514d-c7ee-4019-a26c-700f310ea8e7-0', usage_metadata={'input_tokens': 298, 'output_tokens': 106, 'total_tokens': 404, 'input_token_details': {'cache_read': 0, 'cache_creation': 0}})

In [5166]:
output_formatted = output.content
# turn into json
output = json.loads(output_formatted)
len(output), output

(7,
 {'Common Folk': ['common folk', 'community'],
  'Upper Class Characters': ['Upper class characters', 'upper class'],
  'Supporting Characters': ['Supporting characters',
   'supporting characters',
   'Supportive characters'],
  'Main Characters': ['main characters', 'Main characters'],
  'Romantic Characters': ['Romantic characters'],
  'Other Characters': ['other characters', 'Social characters'],
  'Talking Animals': ['talking animals']})

In [5167]:
# Capitalize output 
capitalized_output = {string.capwords(key): [string.capwords(name) for name in value] for key, value in output.items()}

# Process each scene
for c in characters:

    # Process group
    group = string.capwords(c.group)
    found_group = False

    # Check if the group is a key in the dictionary
    if group in capitalized_output:
        found_group = True
        c.group = group  # Capitalize the group

    # If not found in the dictionary, check the aliases
    if not found_group:
        matched_key = find_key_from_alias(group, capitalized_output)
        if matched_key != group:
            c.group = matched_key
            found_group = True
    if not found_group:
        c.group = group

### assign colors to characters

In [5168]:
# now add colors for top characters
top_char_names = list(top_characters.keys())

all_res_colors = []
invalid_input = True
while invalid_input: # try until valid input
    # see if exception is thrown
    # filter all_json to only include characters in arr
    new_scenes = []
    for scene in scenes:
        include = False
        for character in scene["characters"]:
            if character["name"] in top_char_names:
                include = True
                break
        if include:
            new_scenes.append(scene)

    # take middle 126000 characters of new_scenes
    new_scenes_str = json.dumps(new_scenes)
    new_scenes_str = new_scenes_str[:126000]

    try:
        prompt = f"characters: {top_char_names}\nstory info: {new_scenes_str}\nyour output should contain exactly the same {len(top_char_names)} characters as in the original list. Make sure each character is assigned a unique color."
        res = colorlist_llm.invoke(prompt)
        all_res_colors.append(res)
        invalid_input = False
    except Exception as e:
        print("Exception thrown. Trying again.")
        print("Error:", e)
        invalid_input = True

In [5169]:
# find duplicate colors
colorlist = []
for res in all_res_colors:
    colorlist += res.colorlist
    
unique_colors = []
duplicate_colors = []
for i, char in enumerate(colorlist):
    color = char.color
    if color not in unique_colors:
        unique_colors.append(color)
    else:
        print("duplicate color:", color, i)
        duplicate_colors.append(i)

duplicate color: rgb(255, 215, 0) 17


In [5170]:
colorinfo = {}

for c in colorlist:
    colorinfo[c.name] = {
        "color": c.color,
        "explanation": c.explanation
    }

colorinfo

{'Jo March': {'color': 'rgb(255, 99, 71)',
  'explanation': "This vibrant red represents Jo's passionate and adventurous spirit."},
 'Meg March': {'color': 'rgb(135, 206, 250)',
  'explanation': "A soft blue reflects Meg's nurturing and gentle nature."},
 'Amy March': {'color': 'rgb(255, 182, 193)',
  'explanation': "Light pink symbolizes Amy's youthful vanity and artistic flair."},
 'Theodore Laurence': {'color': 'rgb(60, 179, 113)',
  'explanation': "A fresh green represents Laurie's lively and friendly personality."},
 'Beth March': {'color': 'rgb(255, 228, 196)',
  'explanation': "A warm peach color reflects Beth's gentle and caring disposition."},
 'Mrs. March': {'color': 'rgb(255, 215, 0)',
  'explanation': "Gold symbolizes Mrs. March's warmth and the value she brings to her family."},
 'Mr. Brooke': {'color': 'rgb(128, 0, 128)',
  'explanation': "Purple represents Mr. Brooke's dignified and somewhat distant character."},
 'Hannah': {'color': 'rgb(255, 140, 0)',
  'explanation': 

In [5171]:
# remove duplicates
characters = [char for i, char in enumerate(characters) if i not in duplicate_indices]
print(len(characters))

112


In [5172]:
character_info = []
for c in characters:

    # see if character is in color_info
    name = c.name
    color = ""
    explanation = ""
    if name in colorinfo:
        color = colorinfo[name]["color"]
        explanation = colorinfo[name]["explanation"]
    
    character_info.append({
        "name": name,
        "quote": c.quote,
        "group": c.group,
        "color": color,
        "explanation": explanation
    })

len(character_info), character_info

(112,
 [{'name': 'Jo March',
   'quote': '"Going out for exercise"',
   'group': 'Main Characters',
   'color': 'rgb(255, 99, 71)',
   'explanation': "This vibrant red represents Jo's passionate and adventurous spirit."},
  {'name': 'Meg March',
   'quote': '"I should think two long walks this morning would have been enough!"',
   'group': 'Main Characters',
   'color': 'rgb(135, 206, 250)',
   'explanation': "A soft blue reflects Meg's nurturing and gentle nature."},
  {'name': 'Amy March',
   'quote': '"My only comfort," she said to Meg, with tears in her eyes, "is, that mother don\'t take tucks in my dresses whenever I\'m naughty, as Maria Parks\' mother does."',
   'group': 'Main Characters',
   'color': 'rgb(255, 182, 193)',
   'explanation': "Light pink symbolizes Amy's youthful vanity and artistic flair."},
  {'name': 'Beth March',
   'quote': '"I hope you\'ll have a good night, my poor dear."',
   'group': 'Main Characters',
   'color': 'rgb(255, 228, 196)',
   'explanation': "

In [5173]:
for name in need_to_add:
    character_info.append({
        "name": name,
        "quote": "",
        "group": "misc",
        "color": "",
        "explanation": ""
    })

len(character_info), character_info

(112,
 [{'name': 'Jo March',
   'quote': '"Going out for exercise"',
   'group': 'Main Characters',
   'color': 'rgb(255, 99, 71)',
   'explanation': "This vibrant red represents Jo's passionate and adventurous spirit."},
  {'name': 'Meg March',
   'quote': '"I should think two long walks this morning would have been enough!"',
   'group': 'Main Characters',
   'color': 'rgb(135, 206, 250)',
   'explanation': "A soft blue reflects Meg's nurturing and gentle nature."},
  {'name': 'Amy March',
   'quote': '"My only comfort," she said to Meg, with tears in her eyes, "is, that mother don\'t take tucks in my dresses whenever I\'m naughty, as Maria Parks\' mother does."',
   'group': 'Main Characters',
   'color': 'rgb(255, 182, 193)',
   'explanation': "Light pink symbolizes Amy's youthful vanity and artistic flair."},
  {'name': 'Beth March',
   'quote': '"I hope you\'ll have a good night, my poor dear."',
   'group': 'Main Characters',
   'color': 'rgb(255, 228, 196)',
   'explanation': "

## produce location summaries

In [5174]:
# Pydantic
class LocationInfo(BaseModel):
    """Get information about a location in the story."""
    name: str = Field(description="Name of location.")
    quote: str = Field(description="Direct quote from the story that describes this location")
    emoji: str = Field(description="Emoji that represents this location (e.g., 🏰). Make sure each location has a different emoji.")

class LocationList(BaseModel):
    """Get overall information about the story."""
    locationList: list[LocationInfo] = Field(description="List of location details. One entry per location.")

locationList_llm = llm.with_structured_output(LocationList)

In [5175]:
if not scene_details_exist:
    cropped_json = json.dumps(all_json)[:126000]
    location_prompt = f"locations: {all_locations_new}\nstory info: {cropped_json}\nyour output should contain exactly the same {len(all_locations_new)} locations as in the original list."
    invalid_input = True
    while invalid_input: # try until valid input
        # see if exception is thrown
        try:
            res = locationList_llm.invoke(location_prompt)
            invalid_input = False
        except Exception as e:
            print("Exception thrown. Trying again.")
            print("Error:", e)
            invalid_input = True

In [5176]:
if not scene_details_exist:
    location_info = []
    for l in res.locationList:
        location_info.append({
            "name": l.name,
            "quote": l.quote,
            "emoji": l.emoji
        })

    print(len(location_info))
    print(location_info)

123
[{'name': 'Living Room', 'quote': 'The sisters gathered in the living room, sharing stories and laughter, creating a warm atmosphere.', 'emoji': '🛋️'}, {'name': 'March Family Home', 'quote': 'The March family home was filled with love and warmth, a sanctuary for the sisters.', 'emoji': '🏡'}, {'name': 'Christmas Gathering', 'quote': 'The Christmas gathering was a time of joy, where the sisters exchanged gifts and shared their hopes.', 'emoji': '🎄'}, {'name': 'Garret', 'quote': 'In the garret, Jo found her sanctuary, a place to write and dream.', 'emoji': '🏠'}, {'name': "Mrs. Gardiner's House", 'quote': "Mrs. Gardiner's house was a place of comfort and friendship, where the girls often visited.", 'emoji': '🏡'}, {'name': 'Curtained Recess', 'quote': 'The curtained recess provided a cozy nook for the sisters to share secrets and dreams.', 'emoji': '🪟'}, {'name': 'Party Venue', 'quote': 'The party venue was alive with laughter and music, a celebration of friendship.', 'emoji': '🎉'}, {'n

## generating the final json file!

In [5177]:
if scene_details_exist:
    # load in locations from final_data
    with open(f"json/{og_story_name}/final_data.json") as f:
        location_info = json.load(f)["locations"]
    print("Loaded existing location info")

In [5178]:
# add character and location info to all_json
all_json["characters"] = character_info
all_json["locations"] = location_info
all_json

{'title': 'Little Women',
 'type': 'Book',
 'author': 'Louisa May Alcott',
 'year': '1868',
 'url': 'https://www.gutenberg.org/ebooks/37106',
 'image': 'https://cdn.britannica.com/04/126004-050-EC4DF54F/Dustcover-Louisa-May-Alcott-Little-Women-novel.jpg',
 'num_chapters': 47,
 'num_scenes': 310,
 'num_characters': 112,
 'num_locations': 123,
 'chapters': [{'chapter': 'I. PLAYING PILGRIMS.',
   'scenes': 5,
   'length': 26104,
   'num_lines': 368,
   'summary': "The sisters prioritize their mother's happiness over their own by choosing to buy her gifts, reflecting on their childhood and family traditions.",
   'importance': 0.82,
   'conflict': 0.22,
   'locations': {'Living Room': 2, 'March Family Home': 3},
   'characters': {'Jo March': 5,
    'Meg March': 5,
    'Amy March': 5,
    'Beth March': 5,
    'Mrs. March': 3},
   'links': [{'source': 'Jo March',
     'target': 'Meg March',
     'value': 5,
     'interaction': 'Jo and Meg express their disappointment about not receiving gift

In [5179]:
# save as json
with open(f"json/{story_name}/final_data.json", "w") as f:
    json.dump(all_json, f, indent=4)

# also save to ../src/data/ folder
data_file_name = og_story_name + "-new"
if analysis_type == "theme":
    data_file_name += "-themes"
with open(f"../src/data/{data_file_name}.json", "w") as f:
    json.dump(all_json, f, indent=4)

## compute time taken

In [5180]:
end_time = time.time()
total_time = end_time - start_time

In [5181]:
# convert to minutes
total_time_min = total_time / 60
print(f"Total time taken (minutes): {total_time_min:0.2f}")

Total time taken (minutes): 3.95
